# Дообучение encoder'а для классификации токенов.

Реализация классификатора.

In [ ]:
# Установка зависимостей (при необходимости)
!pip install -q evaluate seqeval transformers datasets

import gc
import math
from typing import List, Dict, Any, Optional, Tuple

import numpy as np
import torch
import evaluate
from transformers import (
    AutoModelForTokenClassification,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
    DataCollatorForTokenClassification,
    TrainerCallback,
    PrinterCallback,
)
from transformers.modeling_outputs import ModelOutput
import pandas as pd
from datasets import Dataset
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm


def set_seed(seed: int = 42):
    """
    Устанавливает фиксированное зерно для Python, NumPy и PyTorch (включая все доступные CUDA-устройства).

    :param seed: Значение зерна.
    """
    import random
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)


class PbarConsoleLogger(TrainerCallback):
    """
    Внешний прогресс‑бар и консольный логгер метрик/лоссов для стабильного отображения на больших данных.
    """
    def __init__(self, pbar):
        self.pbar = pbar
        self.last_logs = {}
        self.last_train_loss = None
        self.printed_eval_steps = set()

    def _step(self, state) -> int:
        return int(state.global_step or 0)

    def _fmt_postfix(self):
        parts = []
        if 'loss' in self.last_logs:
            parts.append(f"loss {self.last_logs['loss']:.4f}")
        if 'eval_loss' in self.last_logs:
            parts.append(f"val {self.last_logs['eval_loss']:.4f}")
        for k, v in self.last_logs.items():
            if k.startswith('eval_') and k not in (
                'eval_loss', 'eval_runtime', 'eval_samples_per_second', 'eval_steps_per_second', 'epoch'
            ):
                try:
                    parts.append(f"{k.replace('eval_', '')} {float(v):.4f}")
                except Exception:
                    pass
        return " | ".join(parts)

    def on_step_end(self, args, state, control, **kwargs):
        n = min(self._step(state), self.pbar.total)
        if n > self.pbar.n:
            self.pbar.update(n - self.pbar.n)
        if self.last_logs:
            self.pbar.set_postfix_str(self._fmt_postfix(), refresh=False)
        self.pbar.refresh()

    def on_log(self, args, state, control, logs=None, **kwargs):
        if not logs:
            return
        for k, v in logs.items():
            if isinstance(v, (int, float)):
                self.last_logs[k] = float(v)
        if 'loss' in logs and isinstance(logs['loss'], (int, float)):
            self.last_train_loss = float(logs['loss'])

        self.pbar.set_postfix_str(self._fmt_postfix(), refresh=False)
        self.pbar.refresh()

        if any(k.startswith('eval_') for k in logs.keys()):
            step = self._step(state)
            if step in self.printed_eval_steps:
                return
            self.printed_eval_steps.add(step)

            train_loss_str = f"{self.last_train_loss:.10f}" if self.last_train_loss is not None else "n/a"
            val_loss = logs.get('eval_loss', None)
            val_loss_str = f"{float(val_loss):.10g}" if isinstance(val_loss, (int, float)) else "n/a"

            exclude = {'eval_loss', 'eval_runtime', 'eval_samples_per_second', 'eval_steps_per_second', 'epoch'}
            extra_parts = []
            for k, v in logs.items():
                if k.startswith('eval_') and k not in exclude:
                    metric_name = k.replace('eval_', '')
                    try:
                        extra_parts.append(f"val {metric_name}: {float(v):.10f}")
                    except Exception:
                        pass

            line = f"step: {step}, train loss: {train_loss_str}, val loss: {val_loss_str}"
            if extra_parts:
                line += ", " + ", ".join(extra_parts)
            tqdm.write(line)

    def on_train_end(self, args, state, control, **kwargs):
        n = min(self._step(state), self.pbar.total)
        if n > self.pbar.n:
            self.pbar.update(n - self.pbar.n)
        self.pbar.refresh()


class WeightedTokenCETrainer(Trainer):
    """
    Кастомный Trainer для токен-классификации с взвешенной CrossEntropy.
    Веса считаются с учетом числа токенов класса (после разметки и чанкинга):
      weight_i = N / (K * n_i),
    где K — число классов, n_i — число токенов класса i, N — сумма всех n_i.
    Отсутствующие классы получают вес 0. Метки -100 игнорируются в потере.

    Поддержка DataParallel: если logits дублируются по числу GPU, метки тайлятся соответствующим образом.
    """
    def __init__(self, *args, class_weights=None, **kwargs):
        # Тихо переводим устаревший аргумент tokenizer в processing_class (совместимость)
        processing = kwargs.pop("tokenizer", None)
        if processing is not None and "processing_class" not in kwargs:
            kwargs["processing_class"] = processing

        super().__init__(*args, **kwargs)

        self.class_weights = None
        if class_weights is not None:
            self.class_weights = torch.as_tensor(class_weights, dtype=torch.float32)
        self._warned_label_tiling = False

    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        """
        Считает взвешенный CrossEntropyLoss по токенам, игнорируя -100.
        Корректно обрабатывает случай DataParallel (дублирование по batch-оси).
        """
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits if hasattr(outputs, "logits") else outputs["logits"]  # [B, L, C]

        # Приводим размеры в соответствие при DataParallel (если нужно)
        if logits.size(0) != labels.size(0):
            ngpu = torch.cuda.device_count()
            if ngpu > 1 and logits.size(0) == labels.size(0) * ngpu:
                labels = labels.repeat_interleave(ngpu, dim=0)
                if not self._warned_label_tiling:
                    print(f"[Warning] DataParallel удвоил batch для logits. "
                          f"Повторяем labels x{ngpu}. logits: {tuple(logits.shape)}, labels: {tuple(labels.shape)}")
                    self._warned_label_tiling = True
            else:
                raise ValueError(f"Batch size mismatch: logits {tuple(logits.shape)} vs labels {tuple(labels.shape)}")

        # Взвешенный CE с ignore_index=-100
        loss_fct = torch.nn.CrossEntropyLoss(
            weight=(self.class_weights.to(logits.device) if self.class_weights is not None else None),
            ignore_index=-100,
        )
        loss = loss_fct(logits.view(-1, logits.size(-1)), labels.view(-1))
        return (loss, outputs) if return_outputs else loss


class TokenClassification:
    """
    Пайплайн токен‑классификации с поддержкой больших данных (чанковое обучение по документам и sliding window).

    Возможности:
      - Автоматическая разметка (sliding window) с перекрытием stride.
      - Взвешенная CrossEntropy по токенам (по всему train) с игнорированием -100.
      - Чанковое обучение: подстановка train_dataset кусками документов (fit_chunk_size_docs).
      - Внешний tqdm прогресс‑бар (стабильный) + консольный лог метрик.

    Необходимые импорты:
    import numpy as np
    import torch
    import evaluate
    from transformers import (
        AutoModelForTokenClassification,
        AutoTokenizer,
        Trainer,
        TrainingArguments,
        DataCollatorForTokenClassification,
    )
    import pandas as pd
    from datasets import Dataset
    from sklearn.model_selection import train_test_split
    from tqdm.auto import tqdm
    """
    def __init__(
        self,
        checkpoint: str,
        label2id: Dict[str, int],
        tokens_column_name: str,
        tags_column_name: str
    ):
        """
        Инициализация модели и токенизатора.

        :param checkpoint: Имя/путь чекпоинта (HF Hub).
        :param label2id: Отображение тегов в id.
        :param tokens_column_name: Имя колонки с токенами (список строк).
        :param tags_column_name: Имя колонки с метками (список тегов или id).
        """
        self.id2label = {v: k for k, v in label2id.items()}
        self.label2id = label2id

        self.model = AutoModelForTokenClassification.from_pretrained(
            checkpoint,
            num_labels=len(self.id2label),
            id2label=self.id2label,
            label2id=self.label2id,
            ignore_mismatched_sizes=True
        )
        self.tokenizer = AutoTokenizer.from_pretrained(checkpoint)
        self.tokens_column_name = tokens_column_name
        self.tags_column_name = tags_column_name

        self.trainer: Optional[Trainer] = None
        self.compute_metrics = None
        self.data_collator = DataCollatorForTokenClassification(tokenizer=self.tokenizer)
        self.progress_callback: Optional[TrainerCallback] = None

    @staticmethod
    def _align_labels_with_word_ids(labels_ids: List[int], word_ids: List[Optional[int]]) -> List[int]:
        """
        Выравнивает метки по word_ids от токенизатора: на первый токен слова ставится метка,
        остальные токены слова получают -100. Спец‑токены (None) получают -100.

        :param labels_ids: Список id‑меток длины = числу слов в документе.
        :param word_ids: Список индексов слов длины = числу токенов в чанке (или None для спец‑токенов).
        :return: Список меток длины = числу токенов в чанке.
        """
        new_labels = []
        prev_word_id = None
        for wid in word_ids:
            if wid is None:
                new_labels.append(-100)
            else:
                if wid != prev_word_id:
                    new_labels.append(labels_ids[wid])
                else:
                    new_labels.append(-100)
            prev_word_id = wid
        return new_labels

    def _tokenize_and_align_chunk(
        self,
        docs_tokens: List[List[str]],
        docs_labels_ids: List[List[int]],
        max_length: int,
        stride: int
    ) -> Dataset:
        """
        Токенизирует список документов (списки токенов) с возвратом переполнений (sliding window),
        выравнивает метки по word_ids, формирует Dataset для обучения/валидации.

        :param docs_tokens: Список документов; каждый документ — список токенов (слов).
        :param docs_labels_ids: Список документов; каждый — список id‑меток по словам.
        :param max_length: Максимальная длина последовательности модели.
        :param stride: Перекрытие при нарезке (число токенов).
        :return: datasets.Dataset с полями input_ids, attention_mask, labels.
        """
        enc = self.tokenizer(
            docs_tokens,
            is_split_into_words=True,
            return_overflowing_tokens=True,
            max_length=max_length,
            stride=stride,
            truncation=True
        )
        mapping = enc.pop("overflow_to_sample_mapping")  # len = кол-во получившихся чанков
        num_chunks = len(enc["input_ids"])

        all_labels = []
        for i in range(num_chunks):
            doc_idx = int(mapping[i])
            word_ids = enc.word_ids(batch_index=i)
            aligned = self._align_labels_with_word_ids(docs_labels_ids[doc_idx], word_ids)
            all_labels.append(aligned)

        return Dataset.from_dict({
            "input_ids": enc["input_ids"],
            "attention_mask": enc["attention_mask"],
            "labels": all_labels
        })

    def _count_total_chunks(
        self,
        docs_tokens: List[List[str]],
        max_length: int,
        stride: int,
        batch_docs: int = 64
    ) -> int:
        """
        Оценивает общее число чанков (последовательностей) после sliding window
        для списка документов, без хранения признаков.

        :param docs_tokens: Список документов (список токенов-слов).
        :param max_length: Максимальная длина последовательности модели.
        :param stride: Перекрытие при нарезке.
        :param batch_docs: Размер пачки документов для ускорения токенизации.
        :return: Общее число последовательностей.
        """
        total = 0
        for i in range(0, len(docs_tokens), batch_docs):
            batch = docs_tokens[i:i + batch_docs]
            enc = self.tokenizer(
                batch,
                is_split_into_words=True,
                return_overflowing_tokens=True,
                max_length=max_length,
                stride=stride,
                truncation=True
            )
            total += len(enc["input_ids"])
        return total

    def _compute_class_weights_over_docs(
        self,
        docs_tokens: List[List[str]],
        docs_labels_ids: List[List[int]],
        max_length: int,
        stride: int,
        batch_docs: int = 32
    ) -> np.ndarray:
        """
        Считает веса классов по ВСЕМ токенам тренировки, выравнивая метки и
        не сохраняя полный датасет в памяти.

        :param docs_tokens: Список документов: список токенов (слов).
        :param docs_labels_ids: Список документов: список id‑меток по словам.
        :param max_length: Максимальная длина последовательности.
        :param stride: Перекрытие при нарезке (в токенах).
        :param batch_docs: Размер батча документов при токенизации.
        :return: Вектор весов классов формы [K].
        """
        num_labels = len(self.id2label)
        counts = np.zeros(num_labels, dtype=np.int64)

        for i in tqdm(range(0, len(docs_tokens), batch_docs), desc="Подсчет частот классов (token-level)"):
            toks = docs_tokens[i:i + batch_docs]
            labs = docs_labels_ids[i:i + batch_docs]

            enc = self.tokenizer(
                toks,
                is_split_into_words=True,
                return_overflowing_tokens=True,
                max_length=max_length,
                stride=stride,
                truncation=True
            )
            mapping = enc.pop("overflow_to_sample_mapping")
            num_chunks = len(enc["input_ids"])

            for j in range(num_chunks):
                doc_idx = int(mapping[j])
                word_ids = enc.word_ids(batch_index=j)
                aligned = self._align_labels_with_word_ids(labs[doc_idx], word_ids)
                arr = np.asarray(aligned, dtype=np.int64)
                arr = arr[arr >= 0]  # игнорируем -100
                if arr.size > 0:
                    bc = np.bincount(arr, minlength=num_labels)
                    counts += bc

        N = counts.sum()
        weights = np.zeros(num_labels, dtype=np.float32)
        nonzero = counts > 0
        if N > 0:
            weights[nonzero] = N / (num_labels * counts[nonzero].astype(np.float32))
        return weights

    def _setup_compute_metrics(self):
        """
        Создает функцию подсчета seqeval-метрик (precision/recall/f1/accuracy).
        """
        metric = evaluate.load("seqeval")

        def compute_seqeval_metrics(p):
            # Поддержка EvalPrediction и tuple
            if isinstance(p, (tuple, list)):
                predictions, labels = p
            else:
                predictions, labels = p.predictions, p.label_ids

            predictions = np.argmax(predictions, axis=2)

            true_predictions = [
                [self.id2label[p] for (p, l) in zip(prediction, label) if l != -100]
                for prediction, label in zip(predictions, labels)
            ]

            true_labels = [
                [self.id2label[l] for (p, l) in zip(prediction, label) if l != -100]
                for prediction, label in zip(predictions, labels)
            ]

            results = metric.compute(predictions=true_predictions, references=true_labels)
            return {
                "precision": results.get("overall_precision", 0.0),
                "recall": results.get("overall_recall", 0.0),
                "f1": results.get("overall_f1", 0.0),
                "accuracy": results.get("overall_accuracy", 0.0),
            }

        self.compute_metrics = compute_seqeval_metrics

    def _prepare_dataset_with_sliding_window(self, df: pd.DataFrame, max_length: int, stride: int) -> Dataset:
        """
        Готовит токенизированный Dataset для списка документов (используется, например, для валидации).
        Выполняет:
          - маппинг строковых тегов -> id при необходимости,
          - токенизацию с переполнениями,
          - выравнивание меток по word_ids.

        :param df: Датафрейм с колонками токенов и меток.
        :param max_length: Максимальная длина модели.
        :param stride: Перекрытие sliding window.
        :return: datasets.Dataset с полями input_ids, attention_mask, labels.
        """
        docs_tokens = df[self.tokens_column_name].tolist()
        docs_labels = df[self.tags_column_name].tolist()

        # Если метки строковые — переводим в id
        if len(docs_labels) and isinstance(docs_labels[0][0], str):
            docs_labels = [[self.label2id[tag] for tag in tags] for tags in docs_labels]

        return self._tokenize_and_align_chunk(docs_tokens, docs_labels, max_length, stride)

    def fit(
        self,
        train_data: pd.DataFrame,
        epochs: int = 3,
        per_device_train_batch_size: int = 16,
        gradient_accumulation_steps: int = 1,
        test_size: float = 0.2,
        learning_rate: float = 2e-5,
        fp16: bool = True,
        stride: int = 128,
        logging_steps: int = 50,
        eval_steps: int = 100,
        output_dir: str = "./result",
        seed: int = 42,
        fit_chunk_size_docs: Optional[int] = None
    ):
        """
        Обучает модель токен‑классификации с поддержкой больших данных:
        train_dataset подставляется чанками документов, внутри которых выполняется sliding window.

        :param train_data: Датафрейм: колонки с токенами и метками на уровне слов.
        :param epochs: Кол-во эпох.
        :param per_device_train_batch_size: Размер батча на устройство.
        :param gradient_accumulation_steps: Шаги аккумуляции градиентов.
        :param test_size: Доля валидации (по документам).
        :param learning_rate: LR для AdamW.
        :param fp16: Использовать fp16 (если bf16 не доступен).
        :param stride: Перекрытие при нарезке (в токенах).
        :param logging_steps: Частота логирования.
        :param eval_steps: Частота валидации/сохранения.
        :param output_dir: Папка для артефактов.
        :param seed: Зерно.
        :param fit_chunk_size_docs: Сколько документов подставлять в один тренировочный чанк. Если None — все.
        :return: self.
        """
        set_seed(seed)
        max_length = int(getattr(self.model.config, "max_position_embeddings", 512))

        # Маппинг строковых тегов -> id (все данные)
        df_all = train_data.copy()
        if len(df_all) and len(df_all[self.tags_column_name]) and isinstance(df_all[self.tags_column_name].iloc[0][0], str):
            df_all[self.tags_column_name] = df_all[self.tags_column_name].apply(
                lambda tags: [self.label2id[tag] for tag in tags]
            )

        # Сплит по документам (потом каждый набор будет чанковаться по sliding window)
        df_train, df_eval = train_test_split(df_all, test_size=test_size, random_state=seed, shuffle=True)

        # Eval датасет можно подготовить целиком (обычно компактнее train)
        eval_dataset = self._prepare_dataset_with_sliding_window(df_eval, max_length, stride)

        # Документы тренировки (списки токенов и меток)
        train_docs_tokens = df_train[self.tokens_column_name].tolist()
        train_docs_labels = df_train[self.tags_column_name].tolist()

        # Веса классов на всём train (по токенам, без хранения полного tokenized train)
        class_weights = self._compute_class_weights_over_docs(
            docs_tokens=train_docs_tokens,
            docs_labels_ids=train_docs_labels,
            max_length=max_length,
            stride=stride,
            batch_docs=32
        )

        # Настройка метрик
        self._setup_compute_metrics()

        # bf16 если доступен, иначе fp16 при флаге
        bf16_ok = bool(torch.cuda.is_available() and torch.cuda.get_device_capability()[0] >= 8)

        args = TrainingArguments(
            output_dir=output_dir,
            num_train_epochs=epochs,
            per_device_train_batch_size=per_device_train_batch_size,
            per_device_eval_batch_size=max(4, per_device_train_batch_size // 2),
            gradient_accumulation_steps=gradient_accumulation_steps,
            eval_accumulation_steps=max(1, gradient_accumulation_steps * 2),
            learning_rate=learning_rate,
            warmup_ratio=0.1,
            lr_scheduler_type="cosine",
            weight_decay=0.01,
            eval_strategy="steps",
            eval_steps=eval_steps,
            save_strategy="steps",
            save_steps=eval_steps,
            load_best_model_at_end=True,
            metric_for_best_model="f1",
            save_total_limit=1,
            logging_strategy="steps",
            logging_steps=logging_steps,
            report_to="none",
            fp16=bool(fp16 and torch.cuda.is_available() and not bf16_ok),
            bf16=bool(bf16_ok and not fp16),
            dataloader_num_workers=0,
            seed=seed,
            remove_unused_columns=False,
            disable_tqdm=True  # используем внешний tqdm
        )

        data_collator = self.data_collator

        # Вспомогательные: шаги по количеству сэмплов (последовательностей), а не документов
        def steps_for_size(n_samples: int, bsz: int, accum: int) -> int:
            """
            Оценивает число оптимизационных шагов на чанке из n_samples последовательностей.
            """
            return max(0, math.ceil(math.ceil(n_samples / max(1, bsz)) / max(1, accum)))

        def chunk_slices(n_docs: int, chunk_docs: int):
            """
            Генератор срезов индексов документов по chunk_docs.
            """
            for i in range(0, n_docs, chunk_docs):
                yield slice(i, min(i + chunk_docs, n_docs))

        # Объем чанка по документам (по умолчанию — все документы)
        n_docs = len(train_docs_tokens)
        chunk_docs = int(fit_chunk_size_docs) if (fit_chunk_size_docs and fit_chunk_size_docs > 0) else n_docs

        # Предварительный расчет total_steps (по числу последовательностей после токенизации)
        total_steps = 0
        rng = np.random.default_rng(seed)
        doc_indices = np.arange(n_docs)

        for _ in range(epochs):
            rng.shuffle(doc_indices)
            for slc in chunk_slices(n_docs, chunk_docs):
                idx = doc_indices[slc]
                toks_chunk = [train_docs_tokens[i] for i in idx]
                # считаем, сколько получится последовательностей в этом чанке документов
                n_samples = self._count_total_chunks(toks_chunk, max_length, stride, batch_docs=64)
                total_steps += steps_for_size(n_samples, per_device_train_batch_size, gradient_accumulation_steps)

        # Инициализируем Trainer с "минимальным" train_dataset (пустой/минимальный чанк)
        # чтобы не держать всю тренировочную выборку
        if n_docs > 0:
            init_chunk_ds = self._tokenize_and_align_chunk(
                [train_docs_tokens[0]], [train_docs_labels[0]], max_length, stride
            )
        else:
            init_chunk_ds = eval_dataset  # fallback

        self.trainer = WeightedTokenCETrainer(
            model=self.model,
            args=args,
            data_collator=data_collator,
            compute_metrics=self.compute_metrics,
            train_dataset=init_chunk_ds,
            eval_dataset=eval_dataset,
            tokenizer=self.tokenizer,
            class_weights=class_weights
        )
        # Удаляем стандартный принтер логов, чтобы не конфликтовал с внешним tqdm
        try:
            self.trainer.remove_callback(PrinterCallback)
        except Exception:
            pass

        # Планировщик под рассчитанное число шагов
        if total_steps > 0:
            self.trainer.create_optimizer_and_scheduler(num_training_steps=total_steps)

        # Внешний прогресс-бар
        pbar = tqdm(total=total_steps, desc="Training Progress", unit="step")
        cb = PbarConsoleLogger(pbar)
        self.trainer.add_callback(cb)
        self.progress_callback = cb

        # Основной цикл обучения по эпохам/чанкам документов
        steps_done = 0
        for ep in range(epochs):
            rng = np.random.default_rng(seed + ep)
            order = np.arange(n_docs)
            rng.shuffle(order)

            for slc in chunk_slices(n_docs, chunk_docs):
                idx = order[slc]
                toks_chunk = [train_docs_tokens[i] for i in idx]
                labs_chunk = [train_docs_labels[i] for i in idx]

                # Готовим датасет последовательностей для этого чанка документов
                ds_chunk = self._tokenize_and_align_chunk(toks_chunk, labs_chunk, max_length, stride)
                self.trainer.train_dataset = ds_chunk

                # Шаги на текущем чанке
                n_samples = len(ds_chunk)  # число последовательностей после sliding window
                chunk_steps = steps_for_size(n_samples, per_device_train_batch_size, gradient_accumulation_steps)
                if chunk_steps == 0:
                    del ds_chunk
                    continue

                # Дообучаем до steps_done + chunk_steps
                self.trainer.args.max_steps = steps_done + chunk_steps
                self.trainer.train()
                steps_done += chunk_steps

                # Очистка памяти
                del ds_chunk
                gc.collect()
                if torch.cuda.is_available():
                    torch.cuda.empty_cache()

        pbar.close()
        return self

    def _predict_single_document(self, tokens: List[str], stride: int) -> List[str]:
        """
        Предсказывает метки на уровне слов для одного документа с помощью sliding window.

        :param tokens: Список токенов (слов) документа.
        :param stride: Перекрытие при нарезке.
        :return: Список предсказанных тегов (строки) длиной = числу слов.
        """
        max_length = int(getattr(self.model.config, "max_position_embeddings", 512))

        tokenized_inputs = self.tokenizer(
            [tokens],
            is_split_into_words=True,
            return_overflowing_tokens=True,
            max_length=max_length,
            stride=stride,
            truncation=True,
        )
        tokenized_inputs.pop("overflow_to_sample_mapping", None)
        chunk_dataset = Dataset.from_dict(tokenized_inputs)

        outputs = self.trainer.predict(chunk_dataset)
        predictions = np.argmax(outputs.predictions, axis=2)

        num_original_words = len(tokens)
        final_predictions = np.full(num_original_words, -1, dtype=np.int32)

        for i, chunk_preds in enumerate(predictions):
            chunk_word_ids = tokenized_inputs.word_ids(batch_index=i)
            for token_pos, word_id in enumerate(chunk_word_ids):
                if word_id is not None and final_predictions[word_id] == -1:
                    final_predictions[word_id] = int(chunk_preds[token_pos])

        return [self.id2label.get(pid, 'O') for pid in final_predictions]

    def predict(self, df: pd.DataFrame, stride: int = 128) -> List[List[str]]:
        """
        Делает предсказания на уровне слов для набора документов (через sliding window).

        :param df: Датафрейм с колонкой токенов (список слов).
        :param stride: Перекрытие при нарезке.
        :return: Список предсказанных последовательностей тегов по документам.
        """
        all_final_labels = []
        for _, row in tqdm(df.iterrows(), total=len(df), desc="Предсказание (sliding window)"):
            original_tokens = row[self.tokens_column_name]
            if not original_tokens:
                all_final_labels.append([])
                continue
            document_labels = self._predict_single_document(original_tokens, stride)
            all_final_labels.append(document_labels)
        return all_final_labels

    def _get_embeddings_single_document(self, tokens: List[str], stride: int, device: torch.device) -> np.ndarray:
        """
        Возвращает усредненные эмбеддинги токенов на уровне слов (после объединения частей
        от sliding window) для одного документа.

        :param tokens: Список токенов (слов).
        :param stride: Перекрытие при нарезке.
        :param device: Целевой девайс модели.
        :return: Массив [num_words, hidden_size].
        """
        max_length = int(getattr(self.model.config, "max_position_embeddings", 512))
        num_original_words = len(tokens)

        chunk_inputs = self.tokenizer(
            [tokens],
            is_split_into_words=True,
            return_overflowing_tokens=True,
            max_length=max_length,
            stride=stride,
            truncation=True,
            return_tensors="pt"
        ).to(device)

        chunk_inputs.pop("overflow_to_sample_mapping")

        with torch.no_grad():
            base_model = getattr(self.trainer.model, self.trainer.model.base_model_prefix)
            outputs = base_model(**chunk_inputs)

        chunk_embeddings = outputs.last_hidden_state  # [num_chunks, seq_len, hidden]

        hidden_size = int(self.model.config.hidden_size)
        final_word_embeddings = torch.zeros(num_original_words, hidden_size, device=device)
        word_counts = torch.zeros(num_original_words, device=device)

        for i in range(len(chunk_embeddings)):
            chunk_embeds = chunk_embeddings[i]
            chunk_word_ids = chunk_inputs.word_ids(batch_index=i)

            for token_pos, word_id in enumerate(chunk_word_ids):
                if word_id is not None:
                    final_word_embeddings[word_id] += chunk_embeds[token_pos]
                    # считаем количество фрагментов для усреднения по слову
                    if token_pos == 0 or chunk_word_ids[token_pos - 1] != word_id:
                        word_counts[word_id] += 1

        average_embeddings = final_word_embeddings / (word_counts.unsqueeze(1) + 1e-8)
        return average_embeddings.detach().cpu().numpy()

    def get_embeddings(self, df: pd.DataFrame, stride: int = 128) -> List[np.ndarray]:
        """
        Генерирует эмбеддинги на уровне слов для набора документов (через sliding window).

        :param df: Датафрейм с колонкой токенов (список слов).
        :param stride: Перекрытие при нарезке.
        :return: Список массивов [num_words, hidden_size] для каждого документа.
        :raises RuntimeError: Если модель не обучена.
        """
        if self.trainer is None or self.trainer.model is None:
            raise RuntimeError("Модель не обучена. Вызовите .fit().")

        self.trainer.model.eval()
        device = self.trainer.model.device
        all_final_embeddings = []

        for _, row in tqdm(df.iterrows(), total=len(df), desc="Генерация эмбеддингов (sliding window)"):
            original_tokens = row[self.tokens_column_name]
            if not original_tokens:
                all_final_embeddings.append(np.zeros((0, int(self.model.config.hidden_size)), dtype=np.float32))
                continue

            document_embeddings = self._get_embeddings_single_document(original_tokens, stride, device)
            all_final_embeddings.append(document_embeddings)

        return all_final_embeddings


Пример использования.

In [ ]:
train_data = pd.DataFrame({
    'tokens': [
        ['Федор', 'Достоевский', 'родился', 'в', 'Москве', '.'],
        ['Анна', 'Керн', 'была', 'музой', 'Пушкина', '.'],
        ['Компания', 'Яндекс', 'представила', 'Алису', '.'],
        ['Илон', 'Маск', 'основал', 'SpaceX', 'и', 'Tesla', '.']
    ],
    'ner_tags': [
        ['B-PER', 'I-PER', 'O', 'O', 'B-LOC', 'O'],
        ['B-PER', 'I-PER', 'O', 'O', 'B-PER', 'O'],
        ['O', 'B-ORG', 'O', 'B-PER', 'O'],
        ['B-PER', 'I-PER', 'O', 'B-ORG', 'O', 'B-ORG', 'O']
    ]
})

# Для предсказания нам нужны только токены
submission_data = pd.DataFrame({
    'tokens': [
        ['Лев', 'Толстой', 'написал', 'роман', '"', 'Война', 'и', 'мир', '"', '.'],
        ['Сергей', 'Королев', 'работал', 'в', 'РКК', '"', 'Энергия', '"', '.']
    ]
})

train_data = pd.concat([train_data] * 15, axis=0)
submission_data = pd.concat([submission_data] * 15, axis=0)

# 2. Создание и обучение модели
# Создаем маппинг из тегов в ID
tags = train_data['ner_tags'].explode().unique()
label2id = {tag: i for i, tag in enumerate(tags)}

model = TokenClassification(
    checkpoint='DeepPavlov/rubert-base-cased',
    label2id=label2id,
    tokens_column_name='tokens',
    tags_column_name='ner_tags'
)

model.fit(
    train_data,
    epochs=3,
    test_size=0.25,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=1,
    learning_rate=2e-5,
    fp16=False,
    logging_steps=10,
    eval_steps=10
)

# 3. Прогнозирование и получение эмбеддингов
labels = model.predict(submission_datap[:5])
embeddings = model.get_embeddings(submission_data[:5])

print(labels)
print(embeddings)

# Дообучение классификатора, который работает с данными разной модальностью.

Реализация классификатора.

In [2]:
!pip install -q wav2clip torchaudio evaluate pillow

import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

import gc
import math
from typing import List, Dict, Any, Optional, Union, Tuple, Callable

import numpy as np
import pandas as pd
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import evaluate
from transformers import Trainer, TrainingArguments, TrainerCallback, PrinterCallback
from transformers.modeling_outputs import SequenceClassifierOutput
from tqdm.auto import tqdm


def set_seed(seed: int = 42):
    """
    Фиксирует зерно для воспроизводимости.

    :param seed: Целое число для инициализации генераторов случайных чисел.
    """
    import random
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)


def to_pil(x: Union[str, np.ndarray, Image.Image]) -> Image.Image:
    """
    Приводит вход к PIL.Image в формате RGB.

    :param x: Путь к изображению, np.ndarray (H,W[,C]) или PIL.Image.
    :return: PIL.Image (RGB).
    :raises ValueError: Если тип входа не поддерживается.
    """
    if isinstance(x, Image.Image):
        return x.convert("RGB")
    if isinstance(x, str):
        return Image.open(x).convert("RGB")
    if isinstance(x, np.ndarray):
        return Image.fromarray(x).convert("RGB")
    raise ValueError("Ожидается путь/np.ndarray/PIL.Image")


def load_audio(path: str, target_sr: int) -> np.ndarray:
    """
    Загружает аудиофайл и при необходимости ресемплирует до target_sr.

    :param path: Путь к файлу (wav/flac и т.п.).
    :param target_sr: Целевая частота дискретизации.
    :return: Одноканальный сигнал формы [T] float32.
    :raises RuntimeError: Если torchaudio не установлен.
    """
    try:
        import torchaudio
    except Exception as e:
        raise RuntimeError("Требуется torchaudio: pip install torchaudio") from e
    waveform, sr = torchaudio.load(path)  # [C, T]
    if waveform.size(0) > 1:
        waveform = waveform.mean(dim=0, keepdim=True)
    if sr != target_sr:
        waveform = torchaudio.functional.resample(waveform, orig_freq=sr, new_freq=target_sr)
    return waveform.squeeze(0).numpy().astype(np.float32)


class MultiComboDataset(Dataset):
    """
    PyTorch Dataset для различных комбинаций модальностей (text/image/audio).
    Поддерживает несколько изображений/аудио per-сэмпл за счёт того, что ячейки могут быть списками.

    Важно: сам датасет хранит лишь ссылки/пути/строки. Реальная загрузка PIL/аудио происходит в collate бэкенда
    (лениво и батчево), что позволяет работать с большими данными.

    :param df: Источник данных (DataFrame).
    :param target_col: Имя колонки с таргетом (классовой меткой).
    :param label2id: Словарь {значение_метки -> id}. Значения меток в df[target_col] должны встречаться в ключах.
    :param text_columns: Текстовые колонки; их значения передаются в tokenizer_fn.
    :param image_columns: Колонки с изображениями (значение — путь/PIL/numpy или список таковых).
    :param audio_columns: Колонки с аудио (значение — путь/массив или список таковых).
    :param text_tokenizer_fn: Функция для токенизации текста. Принимает dict колонок и special_tokens.
    :param special_tokens: Специальные токены для токенизатора.
    """
    def __init__(
        self,
        df: pd.DataFrame,
        target_col: str,
        label2id: Dict[Any, int],
        text_columns: Optional[List[str]] = None,
        image_columns: Optional[List[str]] = None,
        audio_columns: Optional[List[str]] = None,
        text_tokenizer_fn: Optional[Callable] = None,
        special_tokens: Optional[Dict[str, str]] = None
    ):
        self.df = df.reset_index(drop=True)
        self.target_col = target_col
        self.label2id = label2id
        self.text_columns = text_columns or []
        self.image_columns = image_columns or []
        self.audio_columns = audio_columns or []
        self.text_tokenizer_fn = text_tokenizer_fn
        self.special_tokens = special_tokens or {"sep": " [SEP] "}

    def __len__(self) -> int:
        """
        :return: Количество элементов.
        """
        return len(self.df)

    def __getitem__(self, idx: int) -> Dict[str, Any]:
        """
        Возвращает элемент датасета.

        :param idx: Индекс строки.
        :return: Словарь с ключами:
                 - 'labels' (int id)
                 - 'text' (str), если есть текстовые колонки
                 - 'images' (list), если есть колонки картинок
                 - 'audios' (list), если есть колонки аудио
        """
        row = self.df.iloc[idx]
        item = {"labels": int(self.label2id[row[self.target_col]]) if self.target_col in row else 0}

        if self.text_columns:
            if self.text_tokenizer_fn:
                text_data = {c: str(row[c]) if pd.notnull(row[c]) else "" for c in self.text_columns}
                item["text"] = self.text_tokenizer_fn(text_data, self.special_tokens)
            else:
                # Fallback на простую конкатенацию
                sep = self.special_tokens.get("sep", " [SEP] ")
                item["text"] = sep.join([str(row[c]) if pd.notnull(row[c]) else "" for c in self.text_columns])

        def _as_list(v):
            if v is None or (isinstance(v, float) and np.isnan(v)):
                return []
            if isinstance(v, (list, tuple)):
                return list(v)
            return [v]

        if self.image_columns:
            imgs = []
            for c in self.image_columns:
                if c in row:
                    imgs.extend(_as_list(row[c]))
            item["images"] = imgs

        if self.audio_columns:
            auds = []
            for c in self.audio_columns:
                if c in row:
                    auds.extend(_as_list(row[c]))
            item["audios"] = auds

        return item


class BaseBackend(nn.Module):
    """
    Базовый класс мультимодального бэкенда.

    Атрибуты:
      - name: Название бэкенда (str).
      - supported: Набор поддерживаемых модальностей, например {'text','image'}.
      - embed_dim: Базовая размерность эмбеддингов (int).
      - out_dim_per_modality: Реальные выходные размерности (dict modality->int), учитывая агрегацию (concat/mean).
      - text_tokenizer_fn: Функция токенизации текста.
      - special_tokens: Специальные токены для токенизатора.
    """
    name: str = "base"
    supported: set = set()
    embed_dim: int = 0
    out_dim_per_modality: Dict[str, int] = {}
    text_tokenizer_fn: Optional[Callable] = None
    special_tokens: Dict[str, str] = {}

    def collate(self, batch: List[Dict[str, Any]]) -> Dict[str, Any]:
        """
        Собирает батч для Trainer из списка элементов Dataset.

        :param batch: Список элементов (из MultiComboDataset.__getitem__).
        :return: Словарь с 'labels' (LongTensor) и 'backend_inputs' (dict).
        """
        raise NotImplementedError

    def encode(self, backend_inputs: Dict[str, Any], device: torch.device) -> Dict[str, torch.Tensor]:
        """
        Кодирует модальности и возвращает L2-нормированные эмбеддинги.

        :param backend_inputs: Подготовленные входы (collate).
        :param device: Девайс для инференса.
        :return: Словарь {'text':[B,*], 'image':[B,*], 'audio':[B,*]} по доступным модальностям.
        """
        raise NotImplementedError

    def freeze_all(self):
        """
        Замораживает параметры бэкенда (requires_grad=False), полезно для linear probing.
        """
        for p in self.parameters():
            p.requires_grad = False

    def get_out_dim(self, modality: str) -> int:
        """
        Возвращает выходную размерность эмбеддинга по модальности с учётом агрегации.

        :param modality: 'text' | 'image' | 'audio'.
        :return: Размерность вектора.
        """
        return self.out_dim_per_modality.get(modality, self.embed_dim)

    def set_text_tokenizer(self, tokenizer_fn: Optional[Callable], special_tokens: Optional[Dict[str, str]] = None):
        """
        Устанавливает функцию токенизации текста.

        :param tokenizer_fn: Функция токенизации.
        :param special_tokens: Специальные токены.
        """
        self.text_tokenizer_fn = tokenizer_fn
        self.special_tokens = special_tokens or {"sep": " [SEP] "}


class ClipBackend(BaseBackend):
    """
    Бэкенд CLIP (HF) для модальностей: text + image.
    Поддерживает несколько изображений per-сэмпл с агрегацией (concat или mean).

    :param checkpoint: Модель CLIP на HF (например, 'openai/clip-vit-base-patch32').
    :param max_length: Максимальная длина текстовых токенов.
    :param freeze: Заморозить ли веса CLIP.
    :param max_images: Максимум картинок на сэмпл при concat-паде.
    :param image_agg: 'concat' или 'mean' — как агрегировать несколько изображений.
    :param text_tokenizer_fn: Функция токенизации текста.
    :param special_tokens: Специальные токены.
    """
    name = "clip"
    supported = {"text", "image"}

    def __init__(
        self,
        checkpoint: str = "openai/clip-vit-base-patch32",
        max_length: int = 77,
        freeze: bool = True,
        max_images: int = 1,
        image_agg: str = "concat",
        text_tokenizer_fn: Optional[Callable] = None,
        special_tokens: Optional[Dict[str, str]] = None
    ):
        super().__init__()
        from transformers import CLIPModel, CLIPProcessor
        self.model = CLIPModel.from_pretrained(checkpoint)
        self.processor = CLIPProcessor.from_pretrained(checkpoint)
        self.embed_dim = int(self.model.config.projection_dim)
        self.max_length = max_length
        self.max_images = int(max_images)
        self.image_agg = image_agg
        self.text_tokenizer_fn = text_tokenizer_fn
        self.special_tokens = special_tokens or {"sep": " [SEP] "}
        if freeze:
            self.freeze_all()
        img_out = self.embed_dim * self.max_images if self.image_agg == "concat" else self.embed_dim
        self.out_dim_per_modality = {"text": self.embed_dim, "image": img_out}

    def collate(self, batch: List[Dict[str, Any]]) -> Dict[str, Any]:
        """
        Collate для CLIP: ленивая загрузка изображений, подготовка токенов текста.

        :param batch: Список элементов датасета.
        :return: {'labels': LongTensor[B], 'backend_inputs': {...}}
        """
        labels = torch.tensor([b.get("labels", 0) for b in batch], dtype=torch.long)
        texts = [b.get("text", "") for b in batch]

        images_lists = [b.get("images", []) for b in batch]
        flat_images, counts = [], []
        for lst in images_lists:
            lst = lst if isinstance(lst, list) else ([] if lst is None else [lst])
            counts.append(len(lst))
            for img in lst:
                flat_images.append(to_pil(img))

        text_inputs = self.processor(
            text=texts, padding=True, truncation=True,
            max_length=self.max_length, return_tensors="pt"
        )
        text_inputs = {k: v for k, v in text_inputs.items()}

        if len(flat_images):
            img_proc = self.processor(images=flat_images, return_tensors="pt")
            image_inputs = {"pixel_values": img_proc["pixel_values"]}
        else:
            image_inputs = {"pixel_values": None}

        backend_inputs = {
            "text_inputs": text_inputs,
            "image_inputs": image_inputs,
            "image_counts": torch.tensor(counts, dtype=torch.long)
        }
        return {"labels": labels, "backend_inputs": backend_inputs}

    def _concat_padded(self, embs: torch.Tensor, counts: List[int], max_k: int) -> torch.Tensor:
        """
        Конкатенация до max_k эмбеддингов на сэмпл с паддингом нулями.

        :param embs: Плоские эмбеддинги изображений [M, D], где M = сумма counts.
        :param counts: Количество изображений на сэмпл (длина B).
        :param max_k: Максимум картинок на сэмпл.
        :return: [B, D*max_k], L2-нормированный.
        """
        device = embs.device if embs is not None else torch.device("cpu")
        B = len(counts); D = self.embed_dim
        out = torch.zeros((B, D * max_k), device=device, dtype=embs.dtype if embs is not None else torch.float32)
        if embs is None:
            return out
        offset = 0
        for i, c in enumerate(counts):
            if c > 0:
                chunk = embs[offset:offset + c]
                take = chunk[:max_k]
                if take.size(0) < max_k:
                    pad = torch.zeros((max_k - take.size(0), D), device=device, dtype=embs.dtype)
                    take = torch.cat([take, pad], dim=0)
                out[i] = take.reshape(-1)
            offset += c
        return F.normalize(out, dim=-1)

    def _mean_pool(self, embs: torch.Tensor, counts: List[int]) -> torch.Tensor:
        """
        Усреднение эмбеддингов изображений по сэмплу.

        :param embs: Плоские эмбеддинги [M, D].
        :param counts: Количество изображений на сэмпл (длина B).
        :return: [B, D], L2-нормированный.
        """
        device = embs.device if embs is not None else torch.device("cpu")
        B = len(counts); D = self.embed_dim
        out = torch.zeros((B, D), device=device, dtype=embs.dtype if embs is not None else torch.float32)
        if embs is None:
            return out
        offset = 0
        for i, c in enumerate(counts):
            if c > 0:
                out[i] = embs[offset:offset + c].mean(dim=0)
            offset += c
        return F.normalize(out, dim=-1)

    def encode(self, backend_inputs: Dict[str, Any], device: torch.device) -> Dict[str, torch.Tensor]:
        """
        Кодирует текст/изображения через CLIP и агрегирует изображения.

        :param backend_inputs: Выход collate.
        :param device: Девайс.
        :return: {'text':[B,D], 'image':[B, D*max_images] или [B,D]}.
        """
        ti = {k: v.to(device) for k, v in backend_inputs["text_inputs"].items()}
        text_z = self.model.get_text_features(input_ids=ti["input_ids"], attention_mask=ti["attention_mask"])
        text_z = F.normalize(text_z, dim=-1)

        counts = backend_inputs["image_counts"].tolist()
        pi = backend_inputs["image_inputs"]["pixel_values"]
        if pi is not None:
            pi = pi.to(device)
            img_flat = self.model.get_image_features(pixel_values=pi)
            img_flat = F.normalize(img_flat, dim=-1)
            if self.image_agg == "concat":
                img_z = self._concat_padded(img_flat, counts, self.max_images)
            else:
                img_z = self._mean_pool(img_flat, counts)
        else:
            if self.image_agg == "concat":
                img_z = torch.zeros((len(counts), self.embed_dim * self.max_images), device=device)
            else:
                img_z = torch.zeros((len(counts), self.embed_dim), device=device)

        return {"text": text_z, "image": img_z}


class ClapBackend(BaseBackend):
    """
    Бэкенд CLAP (HF) для модальностей: text + audio.
    Поддерживает несколько аудио per-сэмпл (concat/mean).

    :param checkpoint: Модель CLAP (например, 'laion/clap-htsat-unfused').
    :param freeze: Заморозить ли веса CLAP.
    :param max_audios: Максимум аудио на сэмпл при concat-паде.
    :param audio_agg: 'concat' или 'mean' — как агрегировать несколько аудио.
    :param text_tokenizer_fn: Функция токенизации текста.
    :param special_tokens: Специальные токены.
    """
    name = "clap"
    supported = {"text", "audio"}

    def __init__(
        self,
        checkpoint: str = "laion/clap-htsat-unfused",
        freeze: bool = True,
        max_audios: int = 1,
        audio_agg: str = "concat",
        text_tokenizer_fn: Optional[Callable] = None,
        special_tokens: Optional[Dict[str, str]] = None
    ):
        super().__init__()
        from transformers import ClapModel, ClapProcessor
        self.model = ClapModel.from_pretrained(checkpoint)
        self.processor = ClapProcessor.from_pretrained(checkpoint)
        self.embed_dim = int(getattr(self.model.config, "projection_dim", 512))
        sr = getattr(self.processor, "sampling_rate", None)
        if sr is None:
            fe = getattr(self.processor, "feature_extractor", None)
            sr = getattr(fe, "sampling_rate", 48000) if fe is not None else 48000
        self.sr = int(sr)
        self.max_audios = int(max_audios)
        self.audio_agg = audio_agg
        self.text_tokenizer_fn = text_tokenizer_fn
        self.special_tokens = special_tokens or {"sep": " [SEP] "}
        if freeze:
            self.freeze_all()
        aud_out = self.embed_dim * self.max_audios if self.audio_agg == "concat" else self.embed_dim
        self.out_dim_per_modality = {"text": self.embed_dim, "audio": aud_out}

    def collate(self, batch: List[Dict[str, Any]]) -> Dict[str, Any]:
        """
        Collate для CLAP: ленивая загрузка и препроцессинг аудио, токены текста.

        :param batch: Список элементов датасета.
        :return: {'labels': LongTensor[B], 'backend_inputs': {...}}
        """
        labels = torch.tensor([b.get("labels", 0) for b in batch], dtype=torch.long)
        texts = [b.get("text", "") for b in batch]

        audios_lists = [b.get("audios", []) for b in batch]
        flat_audios, counts = [], []
        for lst in audios_lists:
            lst = lst if isinstance(lst, list) else ([] if lst is None else [lst])
            counts.append(len(lst))
            for a in lst:
                if isinstance(a, str):
                    flat_audios.append(load_audio(a, self.sr))
                elif isinstance(a, np.ndarray):
                    flat_audios.append(a.astype(np.float32))
                else:
                    raise ValueError("CLAP ожидает путь к аудио или numpy.ndarray")

        text_inputs = self.processor(text=texts, padding=True, truncation=True, return_tensors="pt")
        text_inputs = {k: v for k, v in text_inputs.items()}

        if len(flat_audios):
            aud_proc = self.processor(audios=flat_audios, sampling_rate=self.sr, padding=True, return_tensors="pt")
            audio_inputs = {"input_features": aud_proc["input_features"]}
        else:
            audio_inputs = {"input_features": None}

        backend_inputs = {
            "text_inputs": text_inputs,
            "audio_inputs": audio_inputs,
            "audio_counts": torch.tensor(counts, dtype=torch.long)
        }
        return {"labels": labels, "backend_inputs": backend_inputs}

    def _concat_padded(self, embs: torch.Tensor, counts: List[int], max_k: int) -> torch.Tensor:
        """
        Конкатенация аудио-эмбеддингов (до max_k) с нулевым паддингом.

        :param embs: Плоские эмбеддинги аудио [M, D].
        :param counts: Кол-во аудио на сэмпл (B).
        :param max_k: Максимум аудио на сэмпл.
        :return: [B, D*max_k], L2-нормированный.
        """
        device = embs.device if embs is not None else torch.device("cpu")
        B = len(counts); D = self.embed_dim
        out = torch.zeros((B, D * max_k), device=device, dtype=embs.dtype if embs is not None else torch.float32)
        if embs is None:
            return out
        offset = 0
        for i, c in enumerate(counts):
            if c > 0:
                chunk = embs[offset:offset + c]
                take = chunk[:max_k]
                if take.size(0) < max_k:
                    pad = torch.zeros((max_k - take.size(0), D), device=device, dtype=embs.dtype)
                    take = torch.cat([take, pad], dim=0)
                out[i] = take.reshape(-1)
            offset += c
        return F.normalize(out, dim=-1)

    def _mean_pool(self, embs: torch.Tensor, counts: List[int]) -> torch.Tensor:
        """
        Усреднение аудио-эмбеддингов по сэмплу.

        :param embs: Плоские эмбеддинги [M, D].
        :param counts: Кол-во аудио на сэмпл (B).
        :return: [B, D], L2-нормированный.
        """
        device = embs.device if embs is not None else torch.device("cpu")
        B = len(counts); D = self.embed_dim
        out = torch.zeros((B, D), device=device, dtype=embs.dtype if embs is not None else torch.float32)
        if embs is None:
            return out
        offset = 0
        for i, c in enumerate(counts):
            if c > 0:
                out[i] = embs[offset:offset + c].mean(dim=0)
            offset += c
        return F.normalize(out, dim=-1)

    def encode(self, backend_inputs: Dict[str, Any], device: torch.device) -> Dict[str, torch.Tensor]:
        """
        Кодирует текст/аудио через CLAP и агрегирует аудио.

        :param backend_inputs: Выход collate.
        :param device: Девайс.
        :return: {'text':[B,D], 'audio':[B, D*max_audios] или [B,D]}.
        """
        ti = {k: v.to(device) for k, v in backend_inputs["text_inputs"].items()}
        text_z = self.model.get_text_features(input_ids=ti["input_ids"], attention_mask=ti["attention_mask"])
        text_z = F.normalize(text_z, dim=-1)

        counts = backend_inputs["audio_counts"].tolist()
        af = backend_inputs["audio_inputs"]["input_features"]
        if af is not None:
            af = af.to(device)
            aud_flat = self.model.get_audio_features(input_features=af)
            aud_flat = F.normalize(aud_flat, dim=-1)
            if self.audio_agg == "concat":
                aud_z = self._concat_padded(aud_flat, counts, self.max_audios)
            else:
                aud_z = self._mean_pool(aud_flat, counts)
        else:
            if self.audio_agg == "concat":
                aud_z = torch.zeros((len(counts), self.embed_dim * self.max_audios), device=device)
            else:
                aud_z = torch.zeros((len(counts), self.embed_dim), device=device)

        return {"text": text_z, "audio": aud_z}


class FlexibleMultiBackend(BaseBackend):
    """
    Гибкий бэкенд, позволяющий использовать разные модели для разных модальностей.
    Поддерживает произвольные комбинации текстовых, визуальных и аудио моделей.

    :param text_model_config: Конфигурация текстовой модели {'checkpoint', 'model_type', 'max_length'}.
    :param image_model_config: Конфигурация визуальной модели {'checkpoint', 'model_type', 'max_images', 'image_agg'}.
    :param audio_model_config: Конфигурация аудио модели {'checkpoint', 'model_type', 'max_audios', 'audio_agg', 'sr'}.
    :param freeze: Заморозить ли веса всех моделей.
    :param text_tokenizer_fn: Функция токенизации текста.
    :param special_tokens: Специальные токены.
    """
    name = "flexible_multi"
    
    def __init__(
        self,
        text_model_config: Optional[Dict[str, Any]] = None,
        image_model_config: Optional[Dict[str, Any]] = None,
        audio_model_config: Optional[Dict[str, Any]] = None,
        freeze: bool = True,
        text_tokenizer_fn: Optional[Callable] = None,
        special_tokens: Optional[Dict[str, str]] = None
    ):
        super().__init__()
        self.supported = set()
        self.out_dim_per_modality = {}
        self.text_tokenizer_fn = text_tokenizer_fn
        self.special_tokens = special_tokens or {"sep": " [SEP] "}
        
        # Инициализация текстовой модели
        self.text_model = None
        self.text_processor = None
        self.text_config = text_model_config or {}
        if text_model_config:
            self._init_text_model(text_model_config)
            self.supported.add("text")
        
        # Инициализация визуальной модели
        self.image_model = None
        self.image_processor = None
        self.image_config = image_model_config or {}
        if image_model_config:
            self._init_image_model(image_model_config)
            self.supported.add("image")
        
        # Инициализация аудио модели
        self.audio_model = None
        self.audio_processor = None
        self.audio_config = audio_model_config or {}
        if audio_model_config:
            self._init_audio_model(audio_model_config)
            self.supported.add("audio")
        
        if freeze:
            self.freeze_all()
    
    def _init_text_model(self, config: Dict[str, Any]):
        """
        Инициализирует текстовую модель согласно конфигурации.
        
        :param config: Словарь с 'checkpoint', 'model_type' и опциональными параметрами.
        """
        from transformers import AutoModel, AutoTokenizer, CLIPTextModel, CLIPTokenizer
        
        checkpoint = config['checkpoint']
        model_type = config.get('model_type', 'auto')
        
        if model_type == 'clip':
            self.text_model = CLIPTextModel.from_pretrained(checkpoint)
            self.text_processor = CLIPTokenizer.from_pretrained(checkpoint)
            # Для CLIP текстовой модели используем projection_dim
            dim = self.text_model.config.projection_dim
        elif model_type == 'bert' or model_type == 'auto':
            self.text_model = AutoModel.from_pretrained(checkpoint)
            self.text_processor = AutoTokenizer.from_pretrained(checkpoint)
            dim = self.text_model.config.hidden_size
        else:
            raise ValueError(f"Неизвестный model_type для текста: {model_type}")
        
        self.text_config['max_length'] = config.get('max_length', 512)
        self.text_config['dim'] = dim
        self.text_config['model_type'] = model_type
        self.out_dim_per_modality['text'] = dim
    
    def _init_image_model(self, config: Dict[str, Any]):
        """
        Инициализирует визуальную модель согласно конфигурации.
        
        :param config: Словарь с 'checkpoint', 'model_type' и опциональными параметрами.
        """
        from transformers import AutoModel, AutoImageProcessor, CLIPVisionModel, CLIPImageProcessor
        
        checkpoint = config['checkpoint']
        model_type = config.get('model_type', 'auto')
        
        if model_type == 'clip':
            self.image_model = CLIPVisionModel.from_pretrained(checkpoint)
            self.image_processor = CLIPImageProcessor.from_pretrained(checkpoint)
            # Для CLIPVisionModel используем hidden_size, так как get_image_features не проецирует
            dim = self.image_model.config.hidden_size
        elif model_type in ['dinov2', 'vit', 'auto']:
            self.image_model = AutoModel.from_pretrained(checkpoint)
            self.image_processor = AutoImageProcessor.from_pretrained(checkpoint)
            dim = self.image_model.config.hidden_size
        else:
            raise ValueError(f"Неизвестный model_type для изображений: {model_type}")
        
        self.image_config['max_images'] = config.get('max_images', 1)
        self.image_config['image_agg'] = config.get('image_agg', 'concat')
        self.image_config['dim'] = dim
        self.image_config['model_type'] = model_type
        
        if self.image_config['image_agg'] == 'concat':
            self.out_dim_per_modality['image'] = dim * self.image_config['max_images']
        else:
            self.out_dim_per_modality['image'] = dim
    
    def _init_audio_model(self, config: Dict[str, Any]):
        """
        Инициализирует аудио модель согласно конфигурации.
        
        :param config: Словарь с 'checkpoint', 'model_type' и опциональными параметрами.
        """
        from transformers import AutoModel, AutoProcessor, ClapAudioModel, ClapProcessor
        
        checkpoint = config['checkpoint']
        model_type = config.get('model_type', 'auto')
        
        if model_type == 'clap':
            from transformers import ClapModel
            self.audio_model = ClapModel.from_pretrained(checkpoint)
            self.audio_processor = ClapProcessor.from_pretrained(checkpoint)
            dim = getattr(self.audio_model.config, "projection_dim", 512)
            sr = getattr(self.audio_processor, "sampling_rate", None)
            if sr is None:
                fe = getattr(self.audio_processor, "feature_extractor", None)
                sr = getattr(fe, "sampling_rate", 48000) if fe is not None else 48000
        elif model_type in ['whisper', 'wav2vec2', 'auto']:
            self.audio_model = AutoModel.from_pretrained(checkpoint)
            self.audio_processor = AutoProcessor.from_pretrained(checkpoint)
            dim = self.audio_model.config.hidden_size
            sr = self.audio_processor.feature_extractor.sampling_rate
        else:
            raise ValueError(f"Неизвестный model_type для аудио: {model_type}")
        
        self.audio_config['sr'] = config.get('sr', sr)
        self.audio_config['max_audios'] = config.get('max_audios', 1)
        self.audio_config['audio_agg'] = config.get('audio_agg', 'concat')
        self.audio_config['dim'] = dim
        self.audio_config['model_type'] = model_type
        
        if self.audio_config['audio_agg'] == 'concat':
            self.out_dim_per_modality['audio'] = dim * self.audio_config['max_audios']
        else:
            self.out_dim_per_modality['audio'] = dim

    def collate(self, batch: List[Dict[str, Any]]) -> Dict[str, Any]:
        """
        Собирает батч для всех активных модальностей с корректной обработкой пропусков.
        """
        labels = torch.tensor([b.get("labels", 0) for b in batch], dtype=torch.long)
        backend_inputs = {}
        batch_size = len(batch)
        
        # Обработка текста
        if self.text_model is not None:
            texts = []
            for b in batch:
                text = b.get("text", "")
                # Если текст пустой или None, используем пробел как заглушку
                texts.append(text if text else " ")
            
            text_inputs = self.text_processor(
                texts, padding=True, truncation=True,
                max_length=self.text_config.get('max_length', 512),
                return_tensors="pt"
            )
            backend_inputs["text_inputs"] = {k: v for k, v in text_inputs.items()}
        
        # Обработка изображений
        if self.image_model is not None:
            images_lists = [b.get("images", []) for b in batch]
            flat_images = []
            img_counts = []
            batch_indices = []  # Отслеживаем к какому сэмплу относится каждое изображение
            
            for idx, lst in enumerate(images_lists):
                lst = lst if isinstance(lst, list) else ([] if lst is None else [lst])
                # Фильтруем None и пустые значения
                lst = [img for img in lst if img is not None]
                img_counts.append(len(lst))
                for img in lst:
                    flat_images.append(to_pil(img))
                    batch_indices.append(idx)
            
            if len(flat_images) > 0:
                img_proc = self.image_processor(images=flat_images, return_tensors="pt")
                backend_inputs["image_inputs"] = {"pixel_values": img_proc["pixel_values"]}
            else:
                backend_inputs["image_inputs"] = {"pixel_values": None}
            
            backend_inputs["image_counts"] = torch.tensor(img_counts, dtype=torch.long)
            backend_inputs["image_batch_indices"] = batch_indices
        
        # Обработка аудио
        if self.audio_model is not None:
            audios_lists = [b.get("audios", []) for b in batch]
            flat_audios = []
            aud_counts = []
            audio_batch_indices = []
            
            for idx, lst in enumerate(audios_lists):
                lst = lst if isinstance(lst, list) else ([] if lst is None else [lst])
                # Фильтруем None и пустые значения
                lst = [a for a in lst if a is not None]
                aud_counts.append(len(lst))
                for a in lst:
                    if isinstance(a, str):
                        flat_audios.append(load_audio(a, self.audio_config['sr']))
                    elif isinstance(a, np.ndarray):
                        flat_audios.append(a.astype(np.float32))
                    audio_batch_indices.append(idx)
            
            if len(flat_audios) > 0:
                if self.audio_config.get('model_type') == 'clap':
                    aud_proc = self.audio_processor(
                        audios=flat_audios, 
                        sampling_rate=self.audio_config['sr'], 
                        padding=True, 
                        return_tensors="pt"
                    )
                    backend_inputs["audio_inputs"] = {"input_features": aud_proc["input_features"]}
                else:
                    aud_proc = self.audio_processor(
                        flat_audios, 
                        sampling_rate=self.audio_config['sr'],
                        padding=True,
                        return_tensors="pt"
                    )
                    backend_inputs["audio_inputs"] = {"input_values": aud_proc["input_values"]}
            else:
                backend_inputs["audio_inputs"] = {"input_features": None, "input_values": None}
            
            backend_inputs["audio_counts"] = torch.tensor(aud_counts, dtype=torch.long)
            backend_inputs["audio_batch_indices"] = audio_batch_indices
        
        backend_inputs["batch_size"] = batch_size
        return {"labels": labels, "backend_inputs": backend_inputs}

    def _aggregate_embeddings(
        self, 
        embs: Optional[torch.Tensor], 
        counts: List[int], 
        max_k: int, 
        dim: int, 
        agg_type: str,
        batch_size: int,
        device: torch.device
    ) -> torch.Tensor:
        """
        Безопасная агрегация эмбеддингов с правильной обработкой пустых сэмплов.
        """
        # Определяем актуальную размерность
        actual_dim = embs.size(1) if embs is not None and embs.numel() > 0 else dim
        
        # Создаем выходной тензор нужного размера
        if agg_type == 'concat':
            out_shape = (batch_size, actual_dim * max_k)
        else:  # mean
            out_shape = (batch_size, actual_dim)
        
        out = torch.zeros(out_shape, device=device, dtype=torch.float32)
        
        # Если нет эмбеддингов, возвращаем нули
        if embs is None or embs.numel() == 0:
            return out
        
        # Агрегируем эмбеддинги для каждого сэмпла
        offset = 0
        for i, count in enumerate(counts):
            if count > 0:
                sample_embs = embs[offset:offset + count]
                
                if agg_type == 'concat':
                    # Берем до max_k эмбеддингов
                    take = sample_embs[:max_k]
                    # Паддинг если нужно
                    if take.size(0) < max_k:
                        pad = torch.zeros((max_k - take.size(0), actual_dim), 
                                        device=device, dtype=embs.dtype)
                        take = torch.cat([take, pad], dim=0)
                    out[i] = take.reshape(-1)
                else:  # mean
                    out[i] = sample_embs.mean(dim=0)
                
                offset += count
        
        return F.normalize(out, dim=-1)
    
    def encode(self, backend_inputs: Dict[str, Any], device: torch.device) -> Dict[str, torch.Tensor]:
        """
        Кодирует все активные модальности через соответствующие модели.
        """
        results = {}
        
        # Определяем актуальный размер батча из labels или первого доступного тензора
        actual_batch_size = None
        
        # Пробуем определить размер батча из различных источников
        if "text_inputs" in backend_inputs:
            for v in backend_inputs["text_inputs"].values():
                if torch.is_tensor(v) and v.dim() > 0:
                    actual_batch_size = v.size(0)
                    break
        
        if actual_batch_size is None and "image_counts" in backend_inputs:
            actual_batch_size = len(backend_inputs["image_counts"])
        
        if actual_batch_size is None and "audio_counts" in backend_inputs:
            actual_batch_size = len(backend_inputs["audio_counts"])
        
        # Если все еще не определен, используем сохраненный
        if actual_batch_size is None:
            actual_batch_size = backend_inputs.get("batch_size", 1)
        
        # Кодирование текста
        if self.text_model is not None and "text_inputs" in backend_inputs:
            text_inputs = {k: v.to(device) for k, v in backend_inputs["text_inputs"].items()}
            
            # Обрезаем входы если нужно (для DataParallel)
            if text_inputs.get("input_ids") is not None:
                current_batch_size = text_inputs["input_ids"].size(0)
                if current_batch_size != actual_batch_size:
                    actual_batch_size = min(actual_batch_size, current_batch_size)
                    text_inputs = {k: v[:actual_batch_size] if torch.is_tensor(v) else v 
                                  for k, v in text_inputs.items()}
            
            if self.text_config.get('model_type') == 'clip':
                text_z = self.text_model.get_text_features(**text_inputs)
            elif hasattr(self.text_model, 'get_text_features'):
                text_z = self.text_model.get_text_features(**text_inputs)
            else:
                outputs = self.text_model(**text_inputs)
                if hasattr(outputs, 'pooler_output') and outputs.pooler_output is not None:
                    text_z = outputs.pooler_output
                else:
                    text_z = outputs.last_hidden_state.mean(dim=1)
            
            results["text"] = F.normalize(text_z, dim=-1)
            # Обновляем actual_batch_size на основе реального выхода
            actual_batch_size = text_z.size(0)
        
        # Кодирование изображений
        if self.image_model is not None and "image_inputs" in backend_inputs:
            pi = backend_inputs["image_inputs"]["pixel_values"]
            counts = backend_inputs["image_counts"]
            
            # Обрезаем counts до actual_batch_size
            if len(counts) > actual_batch_size:
                counts = counts[:actual_batch_size]
            counts = counts.tolist()
            
            # Проверяем, есть ли вообще изображения для обработки
            total_images_needed = sum(counts)
            
            if pi is not None and pi.numel() > 0 and total_images_needed > 0:
                pi = pi.to(device)
                
                # Обрезаем изображения согласно counts
                if pi.size(0) > total_images_needed:
                    pi = pi[:total_images_needed]
                
                if self.image_config.get('model_type') == 'clip':
                    outputs = self.image_model(pixel_values=pi)
                    img_flat = outputs.pooler_output
                elif hasattr(self.image_model, 'get_image_features'):
                    img_flat = self.image_model.get_image_features(pixel_values=pi)
                else:
                    outputs = self.image_model(pixel_values=pi)
                    if hasattr(outputs, 'pooler_output') and outputs.pooler_output is not None:
                        img_flat = outputs.pooler_output
                    else:
                        img_flat = outputs.last_hidden_state[:, 0]
                
                img_flat = F.normalize(img_flat, dim=-1)
                actual_img_dim = img_flat.size(1)
            else:
                img_flat = None
                actual_img_dim = self.image_config.get('dim', 768)
            
            # Используем безопасную агрегацию
            img_z = self._aggregate_embeddings(
                img_flat, counts,
                self.image_config['max_images'],
                actual_img_dim,
                self.image_config['image_agg'],
                len(counts),  # Используем длину counts как размер батча
                device
            )
            
            # Обновляем размерность если изменилась
            if actual_img_dim != self.image_config.get('dim'):
                self.image_config['dim'] = actual_img_dim
                if self.image_config['image_agg'] == 'concat':
                    self.out_dim_per_modality['image'] = actual_img_dim * self.image_config['max_images']
                else:
                    self.out_dim_per_modality['image'] = actual_img_dim
            
            results["image"] = img_z
        
        # Кодирование аудио
        if self.audio_model is not None and "audio_inputs" in backend_inputs:
            counts = backend_inputs["audio_counts"]
            
            # Обрезаем counts до actual_batch_size
            if len(counts) > actual_batch_size:
                counts = counts[:actual_batch_size]
            counts = counts.tolist()
            
            # Проверяем, есть ли вообще аудио для обработки
            total_audios_needed = sum(counts)
            
            # Получаем эмбеддинги аудио
            aud_flat = None
            actual_aud_dim = self.audio_config.get('dim', 768)
            
            if total_audios_needed > 0:  # Обрабатываем только если есть аудио
                if self.audio_config.get('model_type') == 'clap':
                    af = backend_inputs["audio_inputs"]["input_features"]
                    if af is not None and af.numel() > 0:
                        af = af.to(device)
                        
                        # Обрезаем аудио согласно counts
                        if af.size(0) > total_audios_needed:
                            af = af[:total_audios_needed]
                        
                        # Проверяем, что тензор не пустой после обрезки
                        if af.numel() > 0:
                            aud_flat = self.audio_model.get_audio_features(input_features=af)
                            aud_flat = F.normalize(aud_flat, dim=-1)
                            actual_aud_dim = aud_flat.size(1)
                else:
                    av = backend_inputs["audio_inputs"]["input_values"]
                    if av is not None and av.numel() > 0:
                        av = av.to(device)
                        
                        # Обрезаем аудио согласно counts
                        if av.size(0) > total_audios_needed:
                            av = av[:total_audios_needed]
                        
                        # Проверяем, что тензор не пустой после обрезки
                        if av.numel() > 0:
                            outputs = self.audio_model(input_values=av)
                            if hasattr(outputs, 'pooler_output') and outputs.pooler_output is not None:
                                aud_flat = outputs.pooler_output
                            else:
                                aud_flat = outputs.last_hidden_state.mean(dim=1)
                            aud_flat = F.normalize(aud_flat, dim=-1)
                            actual_aud_dim = aud_flat.size(1)
            
            # Используем безопасную агрегацию (она обработает None для aud_flat)
            aud_z = self._aggregate_embeddings(
                aud_flat, counts,
                self.audio_config['max_audios'],
                actual_aud_dim,
                self.audio_config['audio_agg'],
                len(counts),  # Используем длину counts как размер батча
                device
            )
            
            # Обновляем размерность если изменилась
            if aud_flat is not None and actual_aud_dim != self.audio_config.get('dim'):
                self.audio_config['dim'] = actual_aud_dim
                if self.audio_config['audio_agg'] == 'concat':
                    self.out_dim_per_modality['audio'] = actual_aud_dim * self.audio_config['max_audios']
                else:
                    self.out_dim_per_modality['audio'] = actual_aud_dim
            
            results["audio"] = aud_z
        
        # Убеждаемся, что все результаты имеют одинаковый размер батча
        if results:
            min_batch_size = min(v.size(0) for v in results.values())
            if any(v.size(0) != min_batch_size for v in results.values()):
                results = {k: v[:min_batch_size] for k, v in results.items()}
        
        return results


class SingleBackboneClassifier(nn.Module):
    """
    Классификатор поверх одного мультимодального бэкенда: encode -> fuse -> MLP голова.

    :param backend: Экземпляр бэкенда (CLIP/CLAP/FlexibleMultiBackend).
    :param modalities: Активные модальности (учёт порядка важен при concat): подмножество ['image','text','audio'].
    :param num_labels: Количество классов.
    :param fusion: 'concat' (объединение признаков) или 'mean' (среднее по модальностям).
    :param hidden: Размер скрытого слоя головы.
    :param dropout: Дропаут в голове.
    """
    def __init__(
        self,
        backend: BaseBackend,
        modalities: List[str],
        num_labels: int,
        fusion: str = "concat",
        hidden: int = 512,
        dropout: float = 0.1
    ):
        super().__init__()
        self.backend = backend
        self.modalities = modalities
        self.fusion = fusion
        self.num_labels = num_labels

        order = [m for m in ["image", "text", "audio"] if m in self.modalities]
        if fusion == "concat":
            in_dim = sum(self.backend.get_out_dim(m) for m in order)
        elif fusion == "mean":
            dims = [self.backend.get_out_dim(m) for m in order]
            if len(set(dims)) != 1:
                raise ValueError(f'Для fusion="mean" размеры модальностей должны совпадать, а у нас: {dict(zip(order, dims))}')
            in_dim = dims[0]
        else:
            raise ValueError('fusion должен быть "concat" или "mean"')

        self.head = nn.Sequential(
            nn.LayerNorm(in_dim),
            nn.Linear(in_dim, hidden),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden, num_labels)
        )

    def _infer_device_from_inputs(self, obj) -> torch.device:
        """
        Находит девайс по первому тензору во входах; иначе выбирает доступный cuda/cpu.

        :param obj: Любая структура с тензорами.
        :return: torch.device.
        """
        if isinstance(obj, torch.Tensor):
            return obj.device
        if isinstance(obj, dict):
            for v in obj.values():
                d = self._infer_device_from_inputs(v)
                if d is not None:
                    return d
        return torch.device("cuda" if torch.cuda.is_available() else "cpu")

    def _fuse(self, z: Dict[str, torch.Tensor]) -> torch.Tensor:
        """
        Объединяет эмбеддинги модальностей согласно self.fusion.

        :param z: Словарь эмбеддингов по модальностям.
        :return: Fused тензор [B, *].
        """
        order = [m for m in ["image", "text", "audio"] if m in self.modalities]
        feats = []
        batch_size = None
        
        for m in order:
            if m in z:
                t = z[m]
                if t.dim() == 3:
                    t = t.mean(dim=1)
                elif t.dim() > 3:
                    t = t.view(t.size(0), -1)
                feats.append(t)
                if batch_size is None:
                    batch_size = t.size(0)
        
        # Убеждаемся, что все тензоры имеют одинаковый размер батча
        if batch_size is not None:
            feats = [f[:batch_size] for f in feats]
        
        if self.fusion == "concat":
            return torch.cat(feats, dim=-1)
        elif self.fusion == "mean":
            sizes = [f.size(-1) for f in feats]
            if len(set(sizes)) != 1:
                raise ValueError(f'Для fusion="mean" размеры модальностей должны совпадать, а у нас: {sizes}')
            return torch.stack(feats, dim=0).mean(dim=0)

    def forward(self, backend_inputs: Dict[str, Any], labels: Optional[torch.Tensor] = None):
        """
        Прямой проход модели.

        :param backend_inputs: Входы для бэкенда (из его collate).
        :param labels: Игнорируется (loss считает Trainer).
        :return: SequenceClassifierOutput с logits [B, num_labels].
        """
        device = self._infer_device_from_inputs(backend_inputs)
        
        # Кодируем модальности
        z = self.backend.encode(backend_inputs, device=device)
        
        # Проверяем, что получили эмбеддинги
        if not z:
            raise ValueError("Backend не вернул эмбеддинги")
        
        # Объединяем эмбеддинги модальностей
        fused = self._fuse(z)
        
        # Пропускаем через классификационную голову
        logits = self.head(fused)
        
        return SequenceClassifierOutput(logits=logits)

    @torch.no_grad()
    def get_embeddings(self, backend_inputs: Dict[str, Any], return_per_modality: bool = False):
        """
        Извлекает fused эмбеддинги (и опционально по модальностям).

        :param backend_inputs: Входы для бэкенда.
        :param return_per_modality: Вернуть также словарь {'text','image','audio'}.
        :return: fused [B, *] или (fused, per_modality).
        """
        device = self._infer_device_from_inputs(backend_inputs)
        z = self.backend.encode(backend_inputs, device=device)
        fused = self._fuse(z)
        if return_per_modality:
            return fused, z
        return fused


class WeightedCETrainer(Trainer):
    """
    Trainer с CrossEntropyLoss и поддержкой весов классов.
    При отсутствии class_weights может вычислять их по частотам train-меток.

    :param num_labels: Количество классов.
    :param train_labels: Список/массив train-меток (int).
    :param class_weights: Веса классов (list/np.ndarray/torch.Tensor).
    """
    def __init__(self, *args, num_labels=None, train_labels=None, class_weights=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.num_labels = num_labels
        if class_weights is not None:
            w = torch.as_tensor(class_weights, dtype=torch.float32)
        else:
            w = None
            if train_labels is not None and num_labels is not None:
                train_labels = np.asarray(train_labels).astype(int)
                counts = np.bincount(train_labels, minlength=num_labels)
                n = counts.sum()
                weights = np.zeros(num_labels, dtype=np.float32)
                nonzero = counts > 0
                weights[nonzero] = n / (num_labels * counts[nonzero].astype(np.float32))
                w = torch.tensor(weights, dtype=torch.float32)
        self.class_weights = w

    def compute_loss(self, model, inputs, return_outputs: bool = False, num_items_in_batch: Optional[int] = None):
        """
        Считает CrossEntropyLoss с опциональными весами классов. 
        Корректно обрабатывает DataParallel и защищается от NaN.

        :param model: Модель.
        :param inputs: Батч: {'labels': LongTensor[B], 'backend_inputs': {...}}.
        :param return_outputs: Возвращать ли outputs вместе с loss.
        :param num_items_in_batch: Совместимость с Trainer API (не используется).
        :return: loss (и outputs, если return_outputs=True).
        """
        labels = inputs.pop("labels")
        
        # Проверяем, используется ли DataParallel
        is_parallel = isinstance(model, (nn.DataParallel, nn.parallel.DistributedDataParallel))
        
        # Вызываем forward модели
        outputs = model(**inputs)
        logits = outputs.logits
        
        # Переносим labels на устройство logits
        labels = labels.to(logits.device)
        
        # Обработка несоответствия размеров при DataParallel
        if logits.size(0) != labels.size(0):
            # Если logits меньше labels (может быть при разделении батча между GPU)
            if logits.size(0) < labels.size(0):
                # Обрезаем labels до размера logits
                labels = labels[:logits.size(0)]
            # Если logits больше labels (DataParallel может дублировать)
            elif is_parallel:
                # Для DataParallel: повторяем labels для каждой реплики
                num_replicas = logits.size(0) // labels.size(0)
                if logits.size(0) == labels.size(0) * num_replicas:
                    labels = labels.repeat_interleave(num_replicas)
                else:
                    # Если размеры не кратны, берем первые logits.size(0) элементов
                    labels = labels.repeat(num_replicas + 1)[:logits.size(0)]
            else:
                # В других случаях просто обрезаем до минимального размера
                min_size = min(logits.size(0), labels.size(0))
                logits = logits[:min_size]
                labels = labels[:min_size]
        
        # Проверка на NaN и Inf в logits
        if torch.isnan(logits).any() or torch.isinf(logits).any():
            # Заменяем NaN и Inf на нули
            logits = torch.nan_to_num(logits, nan=0.0, posinf=1e4, neginf=-1e4)
        
        # Добавляем небольшую константу для численной стабильности
        eps = 1e-7
        logits = logits + eps
        
        # Вычисляем loss с проверкой
        weight = self.class_weights.to(logits.device) if self.class_weights is not None else None
        
        try:
            loss = nn.CrossEntropyLoss(weight=weight)(logits, labels.long())
            
            # Проверка на NaN в loss
            if torch.isnan(loss) or torch.isinf(loss):
                # Если loss все еще NaN, используем упрощенный вариант без весов
                loss = nn.CrossEntropyLoss()(logits, labels.long())
                
                # Если все еще NaN, возвращаем малое значение
                if torch.isnan(loss) or torch.isinf(loss):
                    loss = torch.tensor(0.01, device=logits.device, requires_grad=True)
        except Exception as e:
            # В случае любой ошибки возвращаем малое значение loss
            print(f"Warning: Error computing loss: {e}")
            loss = torch.tensor(0.01, device=logits.device, requires_grad=True)
        
        return (loss, outputs) if return_outputs else loss


class PbarConsoleLogger(TrainerCallback):
    def __init__(self, pbar):
        self.pbar = pbar
        self.last_logs = {}
        self.last_train_loss = None
        self.printed_eval_steps = set()

    def _step(self, state) -> int:
        return int(state.global_step or 0)

    def _fmt_postfix(self):
        parts = []
        if 'loss' in self.last_logs:
            parts.append(f"loss {self.last_logs['loss']:.4f}")
        if 'eval_loss' in self.last_logs:
            parts.append(f"val {self.last_logs['eval_loss']:.4f}")
        for k, v in self.last_logs.items():
            if k.startswith('eval_') and k not in (
                'eval_loss', 'eval_runtime', 'eval_samples_per_second', 'eval_steps_per_second', 'epoch'
            ):
                parts.append(f"{k.replace('eval_', '')} {v:.4f}")
        return " | ".join(parts)

    def on_step_end(self, args, state, control, **kwargs):
        n = min(self._step(state), self.pbar.total)
        if n > self.pbar.n:
            self.pbar.update(n - self.pbar.n)
        if self.last_logs:
            self.pbar.set_postfix_str(self._fmt_postfix(), refresh=False)
        self.pbar.refresh()

    def on_log(self, args, state, control, logs=None, **kwargs):
        if not logs:
            return

        for k, v in logs.items():
            if isinstance(v, (int, float)):
                self.last_logs[k] = float(v)
        if 'loss' in logs and isinstance(logs['loss'], (int, float)):
            self.last_train_loss = float(logs['loss'])

        self.pbar.set_postfix_str(self._fmt_postfix(), refresh=False)
        self.pbar.refresh()

        if any(k.startswith('eval_') for k in logs.keys()):
            step = self._step(state)
            if step in self.printed_eval_steps:
                return
            self.printed_eval_steps.add(step)

            train_loss_str = f"{self.last_train_loss:.10f}" if self.last_train_loss is not None else "n/a"
            val_loss = logs.get('eval_loss', None)
            val_loss_str = f"{float(val_loss):.10g}" if isinstance(val_loss, (int, float)) else "n/a"

            exclude = {'eval_loss', 'eval_runtime', 'eval_samples_per_second', 'eval_steps_per_second', 'epoch'}
            extra_parts = []
            for k, v in logs.items():
                if k.startswith('eval_') and k not in exclude:
                    metric_name = k.replace('eval_', '')
                    extra_parts.append(f"val {metric_name}: {float(v):.10f}")

            line = f"step: {step}, train loss: {train_loss_str}, val loss: {val_loss_str}"
            if extra_parts:
                line += ", " + ", ".join(extra_parts)
            tqdm.write(line)

    def on_train_end(self, args, state, control, **kwargs):
        n = min(self._step(state), self.pbar.total)
        if n > self.pbar.n:
            self.pbar.update(n - self.pbar.n)
        self.pbar.refresh()


class SingleModelMultiComboClassification:
    """
    Пайплайн: одна мультимодальная модель (бэкенд) + классификационная голова + HuggingFace Trainer.

    Поддерживаемые комбинации модальностей:
      - ['text','image']         -> ClipBackend или FlexibleMultiBackend
      - ['text','audio']         -> ClapBackend или FlexibleMultiBackend
      - ['image','audio']        -> FlexibleMultiBackend
      - ['text','image','audio'] -> FlexibleMultiBackend

    Возможности:
      - Мульти-изображения/аудио per-сэмпл (concat/mean агрегация).
      - Обучение на больших данных: чанковая подстановка train_dataset, стабильный прогресс‑бар и логи.
      - Взвешенная CrossEntropy по частотам классов (для дисбаланса).
      - Гибкая архитектура: возможность использовать разные модели для разных модальностей.
      - Кастомная токенизация текста через переданную функцию.

    Необходимые импорты:
    !pip install -q wav2clip torchaudio evaluate pillow
    import gc
    import math
    from typing import List, Dict, Any, Optional, Union, Tuple, Callable
    import numpy as np
    import pandas as pd
    from PIL import Image
    import torch
    import torch.nn as nn
    import torch.nn.functional as F
    from torch.utils.data import Dataset, DataLoader
    import evaluate
    from transformers import Trainer, TrainingArguments, TrainerCallback, PrinterCallback
    from transformers.modeling_outputs import SequenceClassifierOutput
    from tqdm.auto import tqdm
    """
    def __init__(
        self,
        modalities: List[str],
        num_labels: int,
        target_column_name: str,
        text_columns: Optional[List[str]] = None,
        image_columns: Optional[List[str]] = None,
        audio_columns: Optional[List[str]] = None,
        text_tokenizer_fn: Optional[Callable] = None,
        special_tokens: Optional[Dict[str, str]] = None,
        backend: str = "auto",
        clip_checkpoint: str = "openai/clip-vit-base-patch32",
        clap_checkpoint: str = "laion/clap-htsat-unfused",
        text_model_config: Optional[Dict[str, Any]] = None,
        image_model_config: Optional[Dict[str, Any]] = None,
        audio_model_config: Optional[Dict[str, Any]] = None,
        fusion: str = "concat",
        freeze_backbone: bool = True,
        clip_max_length: int = 77,
        max_images_per_sample: int = 1,
        max_audios_per_sample: int = 1
    ):
        """
        :param modalities: Список модальностей ('text','image','audio') в любом порядке.
        :param num_labels: Количество классов.
        :param target_column_name: Имя столбца таргета в DataFrame.
        :param text_columns: Имена текстовых колонок.
        :param image_columns: Имена колонок изображений (значения — пути или списки путей/объектов).
        :param audio_columns: Имена колонок аудио (значения — пути/массивы или списки).
        :param text_tokenizer_fn: Функция токенизации текста. Принимает dict колонок и special_tokens.
        :param special_tokens: Специальные токены для токенизатора.
        :param backend: 'auto' | 'clip' | 'clap' | 'flexible'.
        :param clip_checkpoint: Чекпоинт CLIP (для backend='clip').
        :param clap_checkpoint: Чекпоинт CLAP (для backend='clap').
        :param text_model_config: Конфиг текстовой модели для FlexibleMultiBackend.
        :param image_model_config: Конфиг визуальной модели для FlexibleMultiBackend.
        :param audio_model_config: Конфиг аудио модели для FlexibleMultiBackend.
        :param fusion: 'concat' или 'mean' — тип фьюжна эмбеддингов.
        :param freeze_backbone: Заморозить веса бэкенда (linear probing).
        :param clip_max_length: Максимальная длина токенов в CLIP.
        :param max_images_per_sample: Максимум картинок при concat-агрегации.
        :param max_audios_per_sample: Максимум аудио при concat-агрегации.
        """
        self.modalities = sorted(list(set(modalities)))
        self.num_labels = num_labels
        self.target_column_name = target_column_name
        self.text_columns = text_columns or []
        self.image_columns = image_columns or []
        self.audio_columns = audio_columns or []
        self.text_tokenizer_fn = text_tokenizer_fn
        self.special_tokens = special_tokens or {"sep": " [SEP] "}
        self.backend_name = backend
        self.clip_checkpoint = clip_checkpoint
        self.clap_checkpoint = clap_checkpoint
        self.text_model_config = text_model_config
        self.image_model_config = image_model_config
        self.audio_model_config = audio_model_config
        self.fusion = fusion
        self.freeze_backbone = freeze_backbone
        self.clip_max_length = clip_max_length
        self.max_images_per_sample = int(max_images_per_sample)
        self.max_audios_per_sample = int(max_audios_per_sample)

        self.label2id: Optional[Dict[Any, int]] = None
        self.id2label: Optional[Dict[int, str]] = None

        self.backend: Optional[BaseBackend] = None
        self.model: Optional[SingleBackboneClassifier] = None
        self.trainer: Optional[Trainer] = None
        self.compute_metrics = None
        self.progress_callback: Optional[PbarConsoleLogger] = None

        self._build_backend()

    def _build_backend(self):
        """
        Инициализирует бэкенд согласно backend='auto'|'clip'|'clap'|'flexible' и проверяет совместимость модальностей.

        :raises ValueError: При неподдерживаемой комбинации модальностей.
        """
        mods = set(self.modalities)
        name = self.backend_name
        if name == "auto":
            if mods == {"text", "image"}:
                name = "clip"
            elif mods == {"text", "audio"}:
                name = "clap"
            else:
                name = "flexible"

        if name == "clip":
            self.backend = ClipBackend(
                checkpoint=self.clip_checkpoint,
                max_length=self.clip_max_length,
                freeze=self.freeze_backbone,
                max_images=self.max_images_per_sample,
                image_agg="concat",
                text_tokenizer_fn=self.text_tokenizer_fn,
                special_tokens=self.special_tokens
            )
        elif name == "clap":
            self.backend = ClapBackend(
                checkpoint=self.clap_checkpoint,
                freeze=self.freeze_backbone,
                max_audios=self.max_audios_per_sample,
                audio_agg="concat",
                text_tokenizer_fn=self.text_tokenizer_fn,
                special_tokens=self.special_tokens
            )
        elif name == "flexible":
            # Автоматическая конфигурация если не заданы модели
            if "text" in mods and self.text_model_config is None:
                self.text_model_config = {
                    'checkpoint': 'bert-base-uncased',
                    'model_type': 'bert',
                    'max_length': 512
                }
            if "image" in mods and self.image_model_config is None:
                self.image_model_config = {
                    'checkpoint': 'google/vit-base-patch16-224',
                    'model_type': 'vit',
                    'max_images': self.max_images_per_sample,
                    'image_agg': 'concat'
                }
            if "audio" in mods and self.audio_model_config is None:
                self.audio_model_config = {
                    'checkpoint': self.clap_checkpoint,
                    'model_type': 'clap',
                    'max_audios': self.max_audios_per_sample,
                    'audio_agg': 'concat',
                    'sr': 48000
                }
            
            self.backend = FlexibleMultiBackend(
                text_model_config=self.text_model_config if "text" in mods else None,
                image_model_config=self.image_model_config if "image" in mods else None,
                audio_model_config=self.audio_model_config if "audio" in mods else None,
                freeze=self.freeze_backbone,
                text_tokenizer_fn=self.text_tokenizer_fn,
                special_tokens=self.special_tokens
            )
        else:
            raise ValueError(f"Неизвестный backend: {name}")

        if not set(self.modalities).issubset(self.backend.supported):
            raise ValueError(f"Бэкенд {self.backend.name} не поддерживает модальности {self.modalities}. "
                             f"Поддерживает: {self.backend.supported}")

    def _setup_metrics(self, metric_name: str):
        """
        Создаёт функцию подсчёта метрик для Trainer.

        :param metric_name: 'f1' или 'accuracy'.
        :raises ValueError: Если метрика не поддерживается.
        """
        metric_name = metric_name.lower()
        if metric_name == "f1":
            metric = evaluate.load("f1")
            def compute(p):
                preds = p.predictions.argmax(-1)
                return metric.compute(predictions=preds, references=p.label_ids, average="weighted")
        elif metric_name == "accuracy":
            metric = evaluate.load("accuracy")
            def compute(p):
                preds = p.predictions.argmax(-1)
                return metric.compute(predictions=preds, references=p.label_ids)
        else:
            raise ValueError('metric_name должен быть "f1" или "accuracy"')
        self.compute_metrics = compute

    def _split(self, df: pd.DataFrame, test_size: float = 0.2, seed: int = 42):
        """
        Перемешивает и делит DataFrame на train/eval.

        :param df: Полный датафрейм.
        :param test_size: Доля валидации (0..1).
        :param seed: Зерно.
        :return: (df_train, df_eval).
        """
        df = df.sample(frac=1.0, random_state=seed).reset_index(drop=True)
        n_eval = int(math.ceil(len(df) * test_size))
        return df.iloc[n_eval:].reset_index(drop=True), df.iloc[:n_eval].reset_index(drop=True)

    def _validate_data(self, df: pd.DataFrame):
        """
        Проверяет соответствие колонок выбранным модальностям.

        :param df: Источник данных.
        :raises ValueError: При отсутствии необходимых колонок.
        """
        if "text" in self.modalities:
            if not self.text_columns:
                raise ValueError("Вы выбрали модальность 'text', но text_columns пустой.")
            missing = [c for c in self.text_columns if c not in df.columns]
            if missing:
                raise ValueError(f"В DataFrame отсутствуют текстовые колонки: {missing}")

        if "image" in self.modalities:
            if not self.image_columns:
                raise ValueError("Вы выбрали модальность 'image', но image_columns пуст.")
            missing = [c for c in self.image_columns if c not in df.columns]
            if missing:
                raise ValueError(f"В DataFrame отсутствуют колонки изображений: {missing}")

        if "audio" in self.modalities:
            if not self.audio_columns:
                raise ValueError("Вы выбрали модальность 'audio', но audio_columns пуст.")
            missing = [c for c in self.audio_columns if c not in df.columns]
            if missing:
                raise ValueError(f"В DataFrame отсутствуют колонки аудио: {missing}")

    def fit(
        self,
        train_data: pd.DataFrame,
        epochs: int = 3,
        test_size: float = 0.2,
        per_device_train_batch_size: int = 16,
        gradient_accumulation_steps: int = 1,
        learning_rate: float = 2e-4,
        metric_name: str = "f1",
        fp16: bool = True,
        logging_steps: int = 50,
        eval_steps: int = 200,
        output_dir: str = "./result",
        seed: int = 42,
        hidden: int = 512,
        dropout: float = 0.1,
        gradient_checkpointing: bool = False,
        fit_chunk_size: Optional[int] = None
    ):
        """
        Обучает классификационную голову поверх выбранного бэкенда.
        Поддерживает обучение на больших данных за счёт чанков: train_dataset подставляется кусками.

        :param train_data: Полный датафрейм с данными и таргетом.
        :param epochs: Количество эпох.
        :param test_size: Доля валидации.
        :param per_device_train_batch_size: Размер батча на устройство.
        :param gradient_accumulation_steps: Шаги аккумуляции градиента.
        :param learning_rate: Learning rate для оптимизатора.
        :param metric_name: 'f1' или 'accuracy' — метрика выбора лучшей модели.
        :param fp16: Использовать fp16 при наличии CUDA (если доступен bf16 — он будет использован вместо fp16).
        :param logging_steps: Частота логирования шагов.
        :param eval_steps: Шаги между валидациями/сохранениями.
        :param output_dir: Каталог для артефактов.
        :param seed: Зерно.
        :param hidden: Размер скрытого слоя головы.
        :param dropout: Дропаут в голове.
        :param gradient_checkpointing: Делать ли чекпоинты во время обучения для экономии VRAM.
        :param fit_chunk_size: Размер чанка обучающей выборки. Если None — весь train как один чанк.
        :return: self.
        """
        self._validate_data(train_data)
        set_seed(seed)

        classes = sorted(train_data[self.target_column_name].unique().tolist())
        if self.num_labels != len(classes):
            print(f"Warning: num_labels={self.num_labels} != len(classes)={len(classes)}")
        self.label2id = {c: i for i, c in enumerate(classes)}
        self.id2label = {i: str(c) for c, i in self.label2id.items()}

        df_train, df_eval = self._split(train_data, test_size=test_size, seed=seed)

        # Датасет валидации держим целиком (обычно небольшой).
        ds_eval = MultiComboDataset(
            df_eval, self.target_column_name, self.label2id,
            self.text_columns, self.image_columns, self.audio_columns,
            self.text_tokenizer_fn, self.special_tokens
        )

        # Веса классов по всему train (не по чанку).
        y_train_all = np.array([self.label2id[y] for y in df_train[self.target_column_name].tolist()], dtype=int)
        counts = np.bincount(y_train_all, minlength=self.num_labels)
        n_all = counts.sum()
        class_weights = np.zeros(self.num_labels, dtype=np.float32)
        nonzero = counts > 0
        class_weights[nonzero] = n_all / (self.num_labels * counts[nonzero].astype(np.float32))

        # Модель и метрики
        self.model = SingleBackboneClassifier(
            backend=self.backend,
            modalities=self.modalities,
            num_labels=self.num_labels,
            fusion=self.fusion,
            hidden=hidden,
            dropout=dropout
        )
        self._setup_metrics(metric_name)

        # Настройки точности
        bf16_ok = bool(torch.cuda.is_available() and torch.cuda.get_device_capability()[0] >= 8)

        args = TrainingArguments(
            output_dir=output_dir,
            num_train_epochs=epochs,
            per_device_train_batch_size=per_device_train_batch_size,
            per_device_eval_batch_size=max(4, per_device_train_batch_size // 2),
            gradient_accumulation_steps=gradient_accumulation_steps,
            eval_accumulation_steps=max(1, gradient_accumulation_steps * 2),
            learning_rate=learning_rate,
            warmup_ratio=0.1,
            lr_scheduler_type="cosine",
            weight_decay=0.01,
            eval_strategy="steps",
            eval_steps=eval_steps,
            save_strategy="steps",
            save_steps=eval_steps,
            load_best_model_at_end=True,
            metric_for_best_model=f"eval_{metric_name}",
            save_total_limit=1,
            logging_strategy="steps",
            logging_steps=logging_steps,
            report_to="none",
            fp16=bool(fp16 and torch.cuda.is_available() and not bf16_ok),
            bf16=bool(bf16_ok and not fp16),
            dataloader_num_workers=os.cpu_count(),
            seed=seed,
            remove_unused_columns=False,
            gradient_checkpointing=gradient_checkpointing,
            dataloader_pin_memory=True,
            ddp_find_unused_parameters=False,
            disable_tqdm=True
        )

        def data_collator(batch_list: List[Dict[str, Any]]) -> Dict[str, Any]:
            """
            Адаптер collate для Trainer: делегирует бэкенду сборку батча.

            :param batch_list: Список элементов Dataset.
            :return: Батч для model.forward(): {'labels': LongTensor, 'backend_inputs': {...}}.
            """
            return self.backend.collate(batch_list)

        # Вспомогательные функции для чанков
        def steps_for_size(sz: int, bsz: int, accum: int) -> int:
            """
            Оценивает число шагов оптимизации на чанке размера sz.

            :param sz: Количество примеров в чанке.
            :param bsz: Размер батча.
            :param accum: Шаги аккумуляции.
            :return: Число оптимизационных шагов.
            """
            return max(0, math.ceil(math.ceil(sz / max(1, bsz)) / max(1, accum)))

        def chunk_slices(index_array: np.ndarray, chunk_size: int):
            """
            Генератор срезов индексов по chunk_size.

            :param index_array: Индексы обучающей выборки.
            :param chunk_size: Размер чанка.
            :yield: Срез индексов.
            """
            for i in range(0, len(index_array), chunk_size):
                yield index_array[i:i + chunk_size]

        # Индексы train
        n_train = len(df_train)
        rng = np.random.default_rng(seed)
        train_idx = np.arange(n_train)

        # Чанк по умолчанию — весь train
        chunk_size = fit_chunk_size if (fit_chunk_size and fit_chunk_size > 0) else len(train_idx)

        # Предварительный рассчёт общего числа шагов (для прогресс‑бара и планировщика)
        total_steps = 0
        for _ in range(epochs):
            rng.shuffle(train_idx)
            for slc in chunk_slices(train_idx, chunk_size):
                total_steps += steps_for_size(len(slc), per_device_train_batch_size, gradient_accumulation_steps)

        # Инициализация Trainer с «пустым» train датасетом (минимальный чанк), чтобы не держать весь train
        dummy_idx = np.arange(min(len(df_train), 1))
        ds_train_init = MultiComboDataset(
            df_train.iloc[dummy_idx], self.target_column_name, self.label2id,
            self.text_columns, self.image_columns, self.audio_columns,
            self.text_tokenizer_fn, self.special_tokens
        ) if len(dummy_idx) > 0 else ds_eval

        self.trainer = WeightedCETrainer(
            model=self.model,
            args=args,
            train_dataset=ds_train_init,
            eval_dataset=ds_eval,
            data_collator=data_collator,
            compute_metrics=self.compute_metrics,
            num_labels=self.num_labels,
            train_labels=y_train_all,
            class_weights=class_weights
        )
        self.trainer.remove_callback(PrinterCallback)

        # Планировщик на рассчитанное количество шагов
        if total_steps > 0:
            self.trainer.create_optimizer_and_scheduler(num_training_steps=total_steps)

        # Внешний прогресс‑бар + консольный лог
        pbar = tqdm(total=total_steps, desc="Training Progress", unit="step")
        cb = PbarConsoleLogger(pbar)
        self.trainer.add_callback(cb)
        self.progress_callback = cb

        # Основной цикл обучения по эпохам и чанкам
        steps_done = 0
        for ep in range(epochs):
            rng = np.random.default_rng(seed + ep)
            shuffled = np.arange(n_train)
            rng.shuffle(shuffled)

            for slc in chunk_slices(shuffled, chunk_size):
                # Подставляем чанк
                chunk_df = df_train.iloc[slc]
                ds_chunk = MultiComboDataset(
                    chunk_df, self.target_column_name, self.label2id,
                    self.text_columns, self.image_columns, self.audio_columns,
                    self.text_tokenizer_fn, self.special_tokens
                )
                self.trainer.train_dataset = ds_chunk

                # Считаем шаги на чанке и настраиваем max_steps Trainer
                chunk_steps = steps_for_size(len(slc), per_device_train_batch_size, gradient_accumulation_steps)
                if chunk_steps == 0:
                    del ds_chunk, chunk_df
                    continue

                self.trainer.args.max_steps = steps_done + chunk_steps
                self.trainer.train()
                steps_done += chunk_steps

                # Очистка памяти между чанками
                del ds_chunk, chunk_df
                gc.collect()
                if torch.cuda.is_available():
                    for i in range(torch.cuda.device_count()):
                        with torch.cuda.device(i):
                            torch.cuda.empty_cache()
                            torch.cuda.ipc_collect()  # Очистка IPC памяти
                    
                    # Синхронизация всех GPU (важно при DataParallel)
                    if torch.cuda.device_count() > 1:
                        for i in range(torch.cuda.device_count()):
                            torch.cuda.synchronize(i)
                    
                    # Дополнительная очистка (если используется)
                    if hasattr(torch.cuda, 'reset_peak_memory_stats'):
                        for i in range(torch.cuda.device_count()):
                            torch.cuda.reset_peak_memory_stats(i)

        pbar.close()
        return self

    def predict(self, df: pd.DataFrame, return_label_str: bool = False) -> np.ndarray:
        """
        Делает предсказания классов на новых данных.

        :param df: Датафрейм с теми же колонками модальностей, что и при обучении.
        :param return_label_str: Если True — вернуть строковые метки; иначе — id.
        :return: np.ndarray предсказанных меток.
        :raises RuntimeError: Если модель ещё не обучена.
        """
        if self.trainer is None:
            raise RuntimeError("Модель не обучена. Вызовите .fit().")
        df_c = df.copy()
        if self.target_column_name not in df_c.columns:
            df_c[self.target_column_name] = list(self.label2id.keys())[0]
        ds = MultiComboDataset(
            df_c, self.target_column_name, self.label2id,
            self.text_columns, self.image_columns, self.audio_columns,
            self.text_tokenizer_fn, self.special_tokens
        )
        preds = self.trainer.predict(test_dataset=ds)
        y_pred = np.argmax(preds.predictions, axis=-1)
        if return_label_str:
            return np.array([self.id2label[int(i)] for i in y_pred])
        return y_pred

    def get_embeddings(self, df: pd.DataFrame, batch_size: int = 32, return_per_modality: bool = False):
        """
        Извлекает fused эмбеддинги (и опционально по модальностям) для новых данных.

        :param df: Датафрейм с нужными колонками модальностей.
        :param batch_size: Размер батча при инференсе.
        :param return_per_modality: Вернуть также словарь эмбеддингов {'text','image','audio'}.
        :return: np.ndarray fused [N, D_fused] или (fused, per_modality_dict).
        :raises RuntimeError: Если модель ещё не обучена.
        """
        if self.trainer is None or self.model is None:
            raise RuntimeError("Модель не обучена. Вызовите .fit().")

        # Определяем девайс модели
        try:
            device = next(self.trainer.model.parameters()).device
        except StopIteration:
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        self.model.to(device).eval()

        df_c = df.copy()
        if self.target_column_name not in df_c.columns:
            df_c[self.target_column_name] = list(self.label2id.keys())[0]

        ds = MultiComboDataset(
            df_c, self.target_column_name, self.label2id,
            self.text_columns, self.image_columns, self.audio_columns,
            self.text_tokenizer_fn, self.special_tokens
        )

        def collate(batch_list: List[Dict[str, Any]]) -> Dict[str, Any]:
            """
            Collate для DataLoader при извлечении эмбеддингов.

            :param batch_list: Список элементов.
            :return: Батч 'backend_inputs' для модели.
            """
            return self.backend.collate(batch_list)

        def move_to_device(obj, device: torch.device):
            """
            Рекурсивно переносит тензоры на device.

            :param obj: Тензор/словарь/список/кортеж/прочее.
            :param device: torch.device.
            :return: Объект с перенесёнными тензорами.
            """
            if torch.is_tensor(obj):
                return obj.to(device)
            if isinstance(obj, dict):
                return {k: move_to_device(v, device) for k, v in obj.items()}
            if isinstance(obj, (list, tuple)):
                t = [move_to_device(v, device) for v in obj]
                return type(obj)(t) if not isinstance(obj, list) else t
            return obj

        loader = DataLoader(ds, batch_size=batch_size, shuffle=False, collate_fn=collate)
        fused_list = []
        per_mod_lists = {m: [] for m in self.modalities} if return_per_modality else None

        with torch.no_grad():
            for batch in loader:
                bi = move_to_device(batch["backend_inputs"], device)
                fused, per = self.model.get_embeddings(backend_inputs=bi, return_per_modality=True)
                fused_list.append(fused.cpu().numpy())
                if return_per_modality:
                    for m in per_mod_lists.keys():
                        if m in per:
                            per_mod_lists[m].append(per[m].cpu().numpy())

        fused_arr = np.vstack(fused_list)
        if not return_per_modality:
            return fused_arr
        per_mod = {m: np.vstack(chunks) for m, chunks in per_mod_lists.items()}
        return fused_arr, per_mod

    def __exit__(self, exc_type, exc_val, exc_tb):
        # Очистка памяти
        if hasattr(self, 'model'):
            del self.model
        if hasattr(self, 'encoders'):
            for encoder in self.encoders.values():
                if hasattr(encoder, 'model'):
                    del encoder.model
        torch.cuda.empty_cache()
        gc.collect()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 77.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 11.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires num

2025-09-02 06:52:04.426743: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756795924.781288      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756795924.881084      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Создание фиктивных данных.

In [3]:
# --- 1. Подготовка и генерация сложных данных ---

# Установка/проверка библиотек
!pip install -q scipy transformers evaluate accelerate

import os
import shutil
import numpy as np
import pandas as pd
from PIL import Image
from scipy.io.wavfile import write as write_wav

def create_complex_dummy_data(
    num_samples: int = 1000,
    class_weights: List[float] = [0.1, 0.6, 0.3] # Несбалансированные классы
) -> pd.DataFrame:
    """Создает сложный фиктивный DataFrame для демонстрации."""
    data_dir = "./complex_test_data"
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)

    labels = ["Documentary", "Music Video", "Sports Review"]
    rng = np.random.default_rng(42)

    # Генерация несбалансированных меток
    generated_labels = rng.choice(labels, num_samples, p=class_weights)

    data = []
    for i, label in enumerate(generated_labels):
        sample = {'id': f'video_{i}'}

        # --- Текстовая модальность ---
        if rng.random() > 0.1: # 10% пропусков текста
            sample['title'] = f"Video Title {i}: A look into {label}"
            sample['transcript'] = f"Welcome to this {label.lower()}. Today we discuss topic {i % 10}."
        else:
            sample['title'] = None
            sample['transcript'] = ""

        # --- Визуальная модальность ---
        if rng.random() > 0.05: # 5% пропусков изображений
            num_images = rng.integers(1, 6) # от 1 до 5 кадров
            img_paths = []
            for j in range(num_images):
                path = os.path.join(data_dir, f"frame_{i}_{j}.png")
                img = Image.fromarray(rng.integers(0, 256, (224, 224, 3), dtype=np.uint8))
                img.save(path)
                img_paths.append(path)
            sample['keyframes'] = img_paths
        else:
            sample['keyframes'] = []

        # --- Аудио модальность ---
        if rng.random() > 0.15: # 15% пропусков аудио
            num_audios = rng.integers(1, 4) # от 1 до 3 аудиоклипов
            audio_paths = []
            for j in range(num_audios):
                path = os.path.join(data_dir, f"sfx_{i}_{j}.wav")
                sr = 48000 # CLAP требует высокую SR
                t = np.linspace(0., 1.5, int(sr * 1.5))
                amplitude = np.iinfo(np.int16).max * 0.3
                freq = rng.uniform(200, 1200)
                waveform = (amplitude * np.sin(2. * np.pi * freq * t)).astype(np.int16)
                write_wav(path, sr, waveform)
                audio_paths.append(path)
            sample['audio_effects'] = audio_paths
        else:
            sample['audio_effects'] = None
            
        sample['genre'] = label
        data.append(sample)

    df = pd.DataFrame(data)
    print(f"Создан DataFrame размером {df.shape}")
    print("Распределение классов:")
    print(df['genre'].value_counts(normalize=True))
    print("\nПример строки:")
    print(df.iloc[rng.integers(0, num_samples)].to_dict())
    return df

# Генерация данных
large_complex_df = create_complex_dummy_data(num_samples=50) # Уменьшено для быстрого запуска

Создан DataFrame размером (50, 6)
Распределение классов:
genre
Music Video      0.58
Sports Review    0.36
Documentary      0.06
Name: proportion, dtype: float64

Пример строки:
{'id': 'video_32', 'title': 'Video Title 32: A look into Music Video', 'transcript': 'Welcome to this music video. Today we discuss topic 2.', 'keyframes': ['./complex_test_data/frame_32_0.png', './complex_test_data/frame_32_1.png', './complex_test_data/frame_32_2.png', './complex_test_data/frame_32_3.png'], 'audio_effects': ['./complex_test_data/sfx_32_0.wav', './complex_test_data/sfx_32_1.wav', './complex_test_data/sfx_32_2.wav'], 'genre': 'Music Video'}


Пример использования.

In [ ]:
video_classifier = SingleModelMultiComboClassification(
    modalities=['text', 'image', 'audio'],
    num_labels=3,
    target_column_name='genre',
    text_columns=['title', 'transcript'],
    image_columns=['keyframes'],
    audio_columns=['audio_effects'],

    backend='flexible',
    text_model_config={
        'checkpoint': 'microsoft/deberta-v3-small',
        'model_type': 'auto',
        'max_length': 128
    },
    image_model_config={
        'checkpoint': 'openai/clip-vit-base-patch32',
        'model_type': 'clip',
        'max_images': 3,
        'image_agg': 'mean'
    },
    audio_model_config={
        'checkpoint': 'laion/clap-htsat-unfused',
        'model_type': 'clap',
        'max_audios': 2,
        'audio_agg': 'mean'
    },

    fusion='concat',
    freeze_backbone=True,
)

video_classifier.fit(
    train_data=large_complex_df,
    epochs=2,
    test_size=0.2,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=5e-6,
    metric_name="f1",
    fp16=True,
    logging_steps=5,
    eval_steps=10,
    output_dir="./video_classifier_results",
    seed=42,
    hidden=512,
    dropout=0.2,
    fit_chunk_size=10
)

inference_df = large_complex_df.sample(5, random_state=42)
predicted_genres = video_classifier.predict(inference_df, return_label_str=True)
print(predicted_genres)

fused_embeddings, per_modality_embeddings = video_classifier.get_embeddings(
    inference_df,
    return_per_modality=True
)
print(f"\nРазмер Fused (объединенных) эмбеддингов: {fused_embeddings.shape}")
print("Размеры эмбеддингов по каждой модальности:")
for modality, embs in per_modality_embeddings.items():
    print(f"  - {modality.capitalize()}: {embs.shape}")

config.json:   0%|          | 0.00/578 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/286M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/286M [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

...

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

Training Progress:   0%|          | 0/24 [00:00<?, ?step/s]

step: 10, train loss: 0.6063000000, val loss: 1.072237968, val f1: 0.4800000000
step: 20, train loss: 0.5039000000, val loss: 1.162315965, val f1: 0.4800000000
['Music Video' 'Music Video' 'Documentary' 'Music Video' 'Music Video']


In [3]:
pipeline = SingleModelMultiComboClassification(
    modalities=["text", "image", "audio"],
    num_labels=4,
    target_column_name="label",

    # Колонки данных
    text_columns=["text_1", "text_2", "text_3"],
    image_columns=["image_path_1", "image_path_2"],
    audio_columns=["audio_path_1", "audio_path_2"],

    # Бэкенд и фьюжн
    backend="auto",     # для ['text','image','audio'] автоматически выберется clip_wav2clip
    fusion="concat",
    freeze_backbone=True,   # linear probing; поставьте False для тонкой донастройки энкодеров

    # Ограничения по количеству мультимодальных входов (для concat-агрегации)
    max_images_per_sample=2,  # под две картинки
    max_audios_per_sample=1    # под одно аудио (если две — выставьте 2 и добавьте вторую колонку)
)

# Обучение (с поддержкой «больших данных» за счёт чанков)
pipeline.fit(
    df_text_image_audio,
    epochs=2,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    metric_name="f1",
    fp16=True,
    logging_steps=10,
    eval_steps=10,
    fit_chunk_size=25,
    output_dir="./result",
    seed=42
)

preds = pipeline.predict(df_text_image_audio[:5], return_label_str=True)
emb = pipeline.get_embeddings(df_text_image_audio[:5], batch_size=8)

print(f'preds: {preds}')
print(f'embeddings: {emb}')

# При желании — эмбеддинги по модальностям отдельно
fused, per_mod = pipeline.get_embeddings(
    df_text_image_audio.head(8),
    batch_size=8,
    return_per_modality=True
)
print("fused.shape:", fused.shape)
for m, arr in per_mod.items():
    print(f"{m} emb shape:", arr.shape)

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Training Progress:   0%|          | 0/52 [00:00<?, ?step/s]

Downloading: "https://github.com/descriptinc/lyrebird-wav2clip/releases/download/v0.1.0-alpha/Wav2CLIP.pt" to /root/.cache/torch/hub/checkpoints/Wav2CLIP.pt
Downloading: "https://github.com/descriptinc/lyrebird-wav2clip/releases/download/v0.1.0-alpha/Wav2CLIP.pt" to /root/.cache/torch/hub/checkpoints/Wav2CLIP.pt

  0%|          | 0.00/46.7M [00:00<?, ?B/s]

  0%|          | 0.00/46.7M [00:00<?, ?B/s]
 16%|█▌        | 7.38M/46.7M [00:00<00:00, 77.1MB/s]

 22%|██▏       | 10.1M/46.7M [00:00<00:00, 100MB/s]
 53%|█████▎    | 24.8M/46.7M [00:00<00:00, 139MB/s] 

100%|██████████| 46.7M/46.7M [00:00<00:00, 165MB/s]

100%|██████████| 46.7M/46.7M [00:00<00:00, 142MB/s]


step: 10, train loss: 1.4042000000, val loss: 1.425664186, val f1: 0.2635167464
step: 20, train loss: 1.2494000000, val loss: 1.705203414, val f1: 0.1702786378
step: 30, train loss: 0.6303000000, val loss: 1.981891394, val f1: 0.2939502603
step: 40, train loss: 0.6404000000, val loss: 2.015444756, val f1: 0.2605555556
step: 50, train loss: 0.9030000000, val loss: 1.774872541, val f1: 0.1573593074
preds: ['B' 'A' 'A' 'C' 'A']
embeddings: [[ 0.00718042 -0.0178346  -0.01379171 ...  0.10906745 -0.02433352
  -0.01562322]
 [-0.00491063 -0.0060696  -0.02768656 ...  0.08318765 -0.00345009
   0.00758308]
 [-0.00189047 -0.01706851 -0.02053576 ...  0.08181054 -0.0128179
  -0.00466248]
 [-0.00369233 -0.01330828 -0.01572154 ...  0.10906745 -0.02433352
  -0.01562322]
 [-0.00369233 -0.01330828 -0.01572154 ...  0.10906745 -0.02433352
  -0.01562322]]
fused.shape: (8, 2048)
audio emb shape: (8, 512)
image emb shape: (8, 1024)
text emb shape: (8, 512)


# Дообучение регрессора, который работает с данными разной модальностью.

Реализация регрессора.

In [1]:
!pip install -q wav2clip torchaudio evaluate pillow

import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

import gc
import math
from typing import List, Dict, Any, Optional, Union, Tuple, Callable

import numpy as np
import pandas as pd
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import evaluate
from transformers import Trainer, TrainingArguments, TrainerCallback, PrinterCallback
from transformers.modeling_outputs import SequenceClassifierOutput
from tqdm.auto import tqdm


def set_seed(seed: int = 42):
    """
    Фиксирует зерно для воспроизводимости.

    :param seed: Целое число для инициализации генераторов случайных чисел.
    """
    import random
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)


def to_pil(x: Union[str, np.ndarray, Image.Image]) -> Image.Image:
    """
    Приводит вход к PIL.Image в формате RGB.

    :param x: Путь к изображению, np.ndarray (H,W[,C]) или PIL.Image.
    :return: PIL.Image (RGB).
    :raises ValueError: Если тип входа не поддерживается.
    """
    if isinstance(x, Image.Image):
        return x.convert("RGB")
    if isinstance(x, str):
        return Image.open(x).convert("RGB")
    if isinstance(x, np.ndarray):
        return Image.fromarray(x).convert("RGB")
    raise ValueError("Ожидается путь/np.ndarray/PIL.Image")


def load_audio(path: str, target_sr: int) -> np.ndarray:
    """
    Загружает аудиофайл и при необходимости ресемплирует до target_sr.

    :param path: Путь к файлу (wav/flac и т.п.).
    :param target_sr: Целевая частота дискретизации.
    :return: Одноканальный сигнал формы [T] float32.
    :raises RuntimeError: Если torchaudio не установлен.
    """
    try:
        import torchaudio
    except Exception as e:
        raise RuntimeError("Требуется torchaudio: pip install torchaudio") from e
    waveform, sr = torchaudio.load(path)  # [C, T]
    if waveform.size(0) > 1:
        waveform = waveform.mean(dim=0, keepdim=True)
    if sr != target_sr:
        waveform = torchaudio.functional.resample(waveform, orig_freq=sr, new_freq=target_sr)
    return waveform.squeeze(0).numpy().astype(np.float32)


class MultiComboRegressionDataset(Dataset):
    """
    PyTorch Dataset для различных комбинаций модальностей (text/image/audio) для регрессии.
    Поддерживает несколько изображений/аудио per-сэмпл за счёт того, что ячейки могут быть списками.
    Поддерживает множественные целевые переменные.

    Важно: сам датасет хранит лишь ссылки/пути/строки. Реальная загрузка PIL/аудио происходит в collate бэкенда
    (лениво и батчево), что позволяет работать с большими данными.

    :param df: Источник данных (DataFrame).
    :param target_columns: Список имён колонок с целевыми значениями.
    :param text_columns: Текстовые колонки; их значения передаются в tokenizer_fn.
    :param image_columns: Колонки с изображениями (значение — путь/PIL/numpy или список таковых).
    :param audio_columns: Колонки с аудио (значение — путь/массив или список таковых).
    :param text_tokenizer_fn: Функция для токенизации текста. Принимает dict колонок и special_tokens.
    :param special_tokens: Специальные токены для токенизатора.
    :param target_normalizer: Опциональная функция нормализации целевых значений.
    """
    def __init__(
        self,
        df: pd.DataFrame,
        target_columns: List[str],
        text_columns: Optional[List[str]] = None,
        image_columns: Optional[List[str]] = None,
        audio_columns: Optional[List[str]] = None,
        text_tokenizer_fn: Optional[Callable] = None,
        special_tokens: Optional[Dict[str, str]] = None,
        target_normalizer: Optional[Callable] = None
    ):
        self.df = df.reset_index(drop=True)
        self.target_columns = target_columns
        self.text_columns = text_columns or []
        self.image_columns = image_columns or []
        self.audio_columns = audio_columns or []
        self.text_tokenizer_fn = text_tokenizer_fn
        self.special_tokens = special_tokens or {"sep": " [SEP] "}
        self.target_normalizer = target_normalizer

    def __len__(self) -> int:
        """
        :return: Количество элементов.
        """
        return len(self.df)

    def __getitem__(self, idx: int) -> Dict[str, Any]:
        """
        Возвращает элемент датасета.

        :param idx: Индекс строки.
        :return: Словарь с ключами:
                 - 'labels' (np.ndarray с целевыми значениями)
                 - 'text' (str), если есть текстовые колонки
                 - 'images' (list), если есть колонки картинок
                 - 'audios' (list), если есть колонки аудио
        """
        row = self.df.iloc[idx]
        
        # Извлекаем целевые значения
        targets = []
        for col in self.target_columns:
            if col in row:
                val = row[col]
                # Обработка NaN значений
                if pd.isna(val):
                    val = 0.0
                targets.append(float(val))
            else:
                targets.append(0.0)
        
        targets = np.array(targets, dtype=np.float32)
        
        # Применяем нормализацию если есть
        if self.target_normalizer is not None:
            targets = self.target_normalizer(targets)
        
        item = {"labels": targets}

        if self.text_columns:
            if self.text_tokenizer_fn:
                text_data = {c: str(row[c]) if pd.notnull(row[c]) else "" for c in self.text_columns}
                item["text"] = self.text_tokenizer_fn(text_data, self.special_tokens)
            else:
                # Fallback на простую конкатенацию
                sep = self.special_tokens.get("sep", " [SEP] ")
                item["text"] = sep.join([str(row[c]) if pd.notnull(row[c]) else "" for c in self.text_columns])

        def _as_list(v):
            if v is None or (isinstance(v, float) and np.isnan(v)):
                return []
            if isinstance(v, (list, tuple)):
                return list(v)
            return [v]

        if self.image_columns:
            imgs = []
            for c in self.image_columns:
                if c in row:
                    imgs.extend(_as_list(row[c]))
            item["images"] = imgs

        if self.audio_columns:
            auds = []
            for c in self.audio_columns:
                if c in row:
                    auds.extend(_as_list(row[c]))
            item["audios"] = auds

        return item


class BaseBackend(nn.Module):
    """
    Базовый класс мультимодального бэкенда.

    Атрибуты:
      - name: Название бэкенда (str).
      - supported: Набор поддерживаемых модальностей, например {'text','image'}.
      - embed_dim: Базовая размерность эмбеддингов (int).
      - out_dim_per_modality: Реальные выходные размерности (dict modality->int), учитывая агрегацию (concat/mean).
      - text_tokenizer_fn: Функция токенизации текста.
      - special_tokens: Специальные токены для токенизатора.
    """
    name: str = "base"
    supported: set = set()
    embed_dim: int = 0
    out_dim_per_modality: Dict[str, int] = {}
    text_tokenizer_fn: Optional[Callable] = None
    special_tokens: Dict[str, str] = {}

    def collate(self, batch: List[Dict[str, Any]]) -> Dict[str, Any]:
        """
        Собирает батч для Trainer из списка элементов Dataset.

        :param batch: Список элементов (из MultiComboRegressionDataset.__getitem__).
        :return: Словарь с 'labels' (FloatTensor) и 'backend_inputs' (dict).
        """
        raise NotImplementedError

    def encode(self, backend_inputs: Dict[str, Any], device: torch.device) -> Dict[str, torch.Tensor]:
        """
        Кодирует модальности и возвращает L2-нормированные эмбеддинги.

        :param backend_inputs: Подготовленные входы (collate).
        :param device: Девайс для инференса.
        :return: Словарь {'text':[B,*], 'image':[B,*], 'audio':[B,*]} по доступным модальностям.
        """
        raise NotImplementedError

    def freeze_all(self):
        """
        Замораживает параметры бэкенда (requires_grad=False), полезно для linear probing.
        """
        for p in self.parameters():
            p.requires_grad = False

    def get_out_dim(self, modality: str) -> int:
        """
        Возвращает выходную размерность эмбеддинга по модальности с учётом агрегации.

        :param modality: 'text' | 'image' | 'audio'.
        :return: Размерность вектора.
        """
        return self.out_dim_per_modality.get(modality, self.embed_dim)

    def set_text_tokenizer(self, tokenizer_fn: Optional[Callable], special_tokens: Optional[Dict[str, str]] = None):
        """
        Устанавливает функцию токенизации текста.

        :param tokenizer_fn: Функция токенизации.
        :param special_tokens: Специальные токены.
        """
        self.text_tokenizer_fn = tokenizer_fn
        self.special_tokens = special_tokens or {"sep": " [SEP] "}


class ClipBackend(BaseBackend):
    """
    Бэкенд CLIP (HF) для модальностей: text + image.
    Поддерживает несколько изображений per-сэмпл с агрегацией (concat или mean).

    :param checkpoint: Модель CLIP на HF (например, 'openai/clip-vit-base-patch32').
    :param max_length: Максимальная длина текстовых токенов.
    :param freeze: Заморозить ли веса CLIP.
    :param max_images: Максимум картинок на сэмпл при concat-паде.
    :param image_agg: 'concat' или 'mean' — как агрегировать несколько изображений.
    :param text_tokenizer_fn: Функция токенизации текста.
    :param special_tokens: Специальные токены.
    """
    name = "clip"
    supported = {"text", "image"}

    def __init__(
        self,
        checkpoint: str = "openai/clip-vit-base-patch32",
        max_length: int = 77,
        freeze: bool = True,
        max_images: int = 1,
        image_agg: str = "concat",
        text_tokenizer_fn: Optional[Callable] = None,
        special_tokens: Optional[Dict[str, str]] = None
    ):
        super().__init__()
        from transformers import CLIPModel, CLIPProcessor
        self.model = CLIPModel.from_pretrained(checkpoint)
        self.processor = CLIPProcessor.from_pretrained(checkpoint)
        self.embed_dim = int(self.model.config.projection_dim)
        self.max_length = max_length
        self.max_images = int(max_images)
        self.image_agg = image_agg
        self.text_tokenizer_fn = text_tokenizer_fn
        self.special_tokens = special_tokens or {"sep": " [SEP] "}
        if freeze:
            self.freeze_all()
        img_out = self.embed_dim * self.max_images if self.image_agg == "concat" else self.embed_dim
        self.out_dim_per_modality = {"text": self.embed_dim, "image": img_out}

    def collate(self, batch: List[Dict[str, Any]]) -> Dict[str, Any]:
        """
        Collate для CLIP: ленивая загрузка изображений, подготовка токенов текста.

        :param batch: Список элементов датасета.
        :return: {'labels': FloatTensor[B, num_targets], 'backend_inputs': {...}}
        """
        labels = torch.tensor(np.stack([b.get("labels", np.array([0.0])) for b in batch]), dtype=torch.float32)
        texts = [b.get("text", "") for b in batch]

        images_lists = [b.get("images", []) for b in batch]
        flat_images, counts = [], []
        for lst in images_lists:
            lst = lst if isinstance(lst, list) else ([] if lst is None else [lst])
            counts.append(len(lst))
            for img in lst:
                flat_images.append(to_pil(img))

        text_inputs = self.processor(
            text=texts, padding=True, truncation=True,
            max_length=self.max_length, return_tensors="pt"
        )
        text_inputs = {k: v for k, v in text_inputs.items()}

        if len(flat_images):
            img_proc = self.processor(images=flat_images, return_tensors="pt")
            image_inputs = {"pixel_values": img_proc["pixel_values"]}
        else:
            image_inputs = {"pixel_values": None}

        backend_inputs = {
            "text_inputs": text_inputs,
            "image_inputs": image_inputs,
            "image_counts": torch.tensor(counts, dtype=torch.long)
        }
        return {"labels": labels, "backend_inputs": backend_inputs}

    def _concat_padded(self, embs: torch.Tensor, counts: List[int], max_k: int) -> torch.Tensor:
        """
        Конкатенация до max_k эмбеддингов на сэмпл с паддингом нулями.

        :param embs: Плоские эмбеддинги изображений [M, D], где M = сумма counts.
        :param counts: Количество изображений на сэмпл (длина B).
        :param max_k: Максимум картинок на сэмпл.
        :return: [B, D*max_k], L2-нормированный.
        """
        device = embs.device if embs is not None else torch.device("cpu")
        B = len(counts); D = self.embed_dim
        out = torch.zeros((B, D * max_k), device=device, dtype=embs.dtype if embs is not None else torch.float32)
        if embs is None:
            return out
        offset = 0
        for i, c in enumerate(counts):
            if c > 0:
                chunk = embs[offset:offset + c]
                take = chunk[:max_k]
                if take.size(0) < max_k:
                    pad = torch.zeros((max_k - take.size(0), D), device=device, dtype=embs.dtype)
                    take = torch.cat([take, pad], dim=0)
                out[i] = take.reshape(-1)
            offset += c
        return F.normalize(out, dim=-1)

    def _mean_pool(self, embs: torch.Tensor, counts: List[int]) -> torch.Tensor:
        """
        Усреднение эмбеддингов изображений по сэмплу.

        :param embs: Плоские эмбеддинги [M, D].
        :param counts: Количество изображений на сэмпл (длина B).
        :return: [B, D], L2-нормированный.
        """
        device = embs.device if embs is not None else torch.device("cpu")
        B = len(counts); D = self.embed_dim
        out = torch.zeros((B, D), device=device, dtype=embs.dtype if embs is not None else torch.float32)
        if embs is None:
            return out
        offset = 0
        for i, c in enumerate(counts):
            if c > 0:
                out[i] = embs[offset:offset + c].mean(dim=0)
            offset += c
        return F.normalize(out, dim=-1)

    def encode(self, backend_inputs: Dict[str, Any], device: torch.device) -> Dict[str, torch.Tensor]:
        """
        Кодирует текст/изображения через CLIP и агрегирует изображения.

        :param backend_inputs: Выход collate.
        :param device: Девайс.
        :return: {'text':[B,D], 'image':[B, D*max_images] или [B,D]}.
        """
        ti = {k: v.to(device) for k, v in backend_inputs["text_inputs"].items()}
        text_z = self.model.get_text_features(input_ids=ti["input_ids"], attention_mask=ti["attention_mask"])
        text_z = F.normalize(text_z, dim=-1)

        counts = backend_inputs["image_counts"].tolist()
        pi = backend_inputs["image_inputs"]["pixel_values"]
        if pi is not None:
            pi = pi.to(device)
            img_flat = self.model.get_image_features(pixel_values=pi)
            img_flat = F.normalize(img_flat, dim=-1)
            if self.image_agg == "concat":
                img_z = self._concat_padded(img_flat, counts, self.max_images)
            else:
                img_z = self._mean_pool(img_flat, counts)
        else:
            if self.image_agg == "concat":
                img_z = torch.zeros((len(counts), self.embed_dim * self.max_images), device=device)
            else:
                img_z = torch.zeros((len(counts), self.embed_dim), device=device)

        return {"text": text_z, "image": img_z}


class ClapBackend(BaseBackend):
    """
    Бэкенд CLAP (HF) для модальностей: text + audio.
    Поддерживает несколько аудио per-сэмпл (concat/mean).

    :param checkpoint: Модель CLAP (например, 'laion/clap-htsat-unfused').
    :param freeze: Заморозить ли веса CLAP.
    :param max_audios: Максимум аудио на сэмпл при concat-паде.
    :param audio_agg: 'concat' или 'mean' — как агрегировать несколько аудио.
    :param text_tokenizer_fn: Функция токенизации текста.
    :param special_tokens: Специальные токены.
    """
    name = "clap"
    supported = {"text", "audio"}

    def __init__(
        self,
        checkpoint: str = "laion/clap-htsat-unfused",
        freeze: bool = True,
        max_audios: int = 1,
        audio_agg: str = "concat",
        text_tokenizer_fn: Optional[Callable] = None,
        special_tokens: Optional[Dict[str, str]] = None
    ):
        super().__init__()
        from transformers import ClapModel, ClapProcessor
        self.model = ClapModel.from_pretrained(checkpoint)
        self.processor = ClapProcessor.from_pretrained(checkpoint)
        self.embed_dim = int(getattr(self.model.config, "projection_dim", 512))
        sr = getattr(self.processor, "sampling_rate", None)
        if sr is None:
            fe = getattr(self.processor, "feature_extractor", None)
            sr = getattr(fe, "sampling_rate", 48000) if fe is not None else 48000
        self.sr = int(sr)
        self.max_audios = int(max_audios)
        self.audio_agg = audio_agg
        self.text_tokenizer_fn = text_tokenizer_fn
        self.special_tokens = special_tokens or {"sep": " [SEP] "}
        if freeze:
            self.freeze_all()
        aud_out = self.embed_dim * self.max_audios if self.audio_agg == "concat" else self.embed_dim
        self.out_dim_per_modality = {"text": self.embed_dim, "audio": aud_out}

    def collate(self, batch: List[Dict[str, Any]]) -> Dict[str, Any]:
        """
        Collate для CLAP: ленивая загрузка и препроцессинг аудио, токены текста.

        :param batch: Список элементов датасета.
        :return: {'labels': FloatTensor[B, num_targets], 'backend_inputs': {...}}
        """
        labels = torch.tensor(np.stack([b.get("labels", np.array([0.0])) for b in batch]), dtype=torch.float32)
        texts = [b.get("text", "") for b in batch]

        audios_lists = [b.get("audios", []) for b in batch]
        flat_audios, counts = [], []
        for lst in audios_lists:
            lst = lst if isinstance(lst, list) else ([] if lst is None else [lst])
            counts.append(len(lst))
            for a in lst:
                if isinstance(a, str):
                    flat_audios.append(load_audio(a, self.sr))
                elif isinstance(a, np.ndarray):
                    flat_audios.append(a.astype(np.float32))
                else:
                    raise ValueError("CLAP ожидает путь к аудио или numpy.ndarray")

        text_inputs = self.processor(text=texts, padding=True, truncation=True, return_tensors="pt")
        text_inputs = {k: v for k, v in text_inputs.items()}

        if len(flat_audios):
            aud_proc = self.processor(audios=flat_audios, sampling_rate=self.sr, padding=True, return_tensors="pt")
            audio_inputs = {"input_features": aud_proc["input_features"]}
        else:
            audio_inputs = {"input_features": None}

        backend_inputs = {
            "text_inputs": text_inputs,
            "audio_inputs": audio_inputs,
            "audio_counts": torch.tensor(counts, dtype=torch.long)
        }
        return {"labels": labels, "backend_inputs": backend_inputs}

    def _concat_padded(self, embs: torch.Tensor, counts: List[int], max_k: int) -> torch.Tensor:
        """
        Конкатенация аудио-эмбеддингов (до max_k) с нулевым паддингом.

        :param embs: Плоские эмбеддинги аудио [M, D].
        :param counts: Кол-во аудио на сэмпл (B).
        :param max_k: Максимум аудио на сэмпл.
        :return: [B, D*max_k], L2-нормированный.
        """
        device = embs.device if embs is not None else torch.device("cpu")
        B = len(counts); D = self.embed_dim
        out = torch.zeros((B, D * max_k), device=device, dtype=embs.dtype if embs is not None else torch.float32)
        if embs is None:
            return out
        offset = 0
        for i, c in enumerate(counts):
            if c > 0:
                chunk = embs[offset:offset + c]
                take = chunk[:max_k]
                if take.size(0) < max_k:
                    pad = torch.zeros((max_k - take.size(0), D), device=device, dtype=embs.dtype)
                    take = torch.cat([take, pad], dim=0)
                out[i] = take.reshape(-1)
            offset += c
        return F.normalize(out, dim=-1)

    def _mean_pool(self, embs: torch.Tensor, counts: List[int]) -> torch.Tensor:
        """
        Усреднение аудио-эмбеддингов по сэмплу.

        :param embs: Плоские эмбеддинги [M, D].
        :param counts: Кол-во аудио на сэмпл (B).
        :return: [B, D], L2-нормированный.
        """
        device = embs.device if embs is not None else torch.device("cpu")
        B = len(counts); D = self.embed_dim
        out = torch.zeros((B, D), device=device, dtype=embs.dtype if embs is not None else torch.float32)
        if embs is None:
            return out
        offset = 0
        for i, c in enumerate(counts):
            if c > 0:
                out[i] = embs[offset:offset + c].mean(dim=0)
            offset += c
        return F.normalize(out, dim=-1)

    def encode(self, backend_inputs: Dict[str, Any], device: torch.device) -> Dict[str, torch.Tensor]:
        """
        Кодирует текст/аудио через CLAP и агрегирует аудио.

        :param backend_inputs: Выход collate.
        :param device: Девайс.
        :return: {'text':[B,D], 'audio':[B, D*max_audios] или [B,D]}.
        """
        ti = {k: v.to(device) for k, v in backend_inputs["text_inputs"].items()}
        text_z = self.model.get_text_features(input_ids=ti["input_ids"], attention_mask=ti["attention_mask"])
        text_z = F.normalize(text_z, dim=-1)

        counts = backend_inputs["audio_counts"].tolist()
        af = backend_inputs["audio_inputs"]["input_features"]
        if af is not None:
            af = af.to(device)
            aud_flat = self.model.get_audio_features(input_features=af)
            aud_flat = F.normalize(aud_flat, dim=-1)
            if self.audio_agg == "concat":
                aud_z = self._concat_padded(aud_flat, counts, self.max_audios)
            else:
                aud_z = self._mean_pool(aud_flat, counts)
        else:
            if self.audio_agg == "concat":
                aud_z = torch.zeros((len(counts), self.embed_dim * self.max_audios), device=device)
            else:
                aud_z = torch.zeros((len(counts), self.embed_dim), device=device)

        return {"text": text_z, "audio": aud_z}


class FlexibleMultiBackend(BaseBackend):
    """
    Гибкий бэкенд, позволяющий использовать разные модели для разных модальностей.
    Поддерживает произвольные комбинации текстовых, визуальных и аудио моделей.

    :param text_model_config: Конфигурация текстовой модели {'checkpoint', 'model_type', 'max_length'}.
    :param image_model_config: Конфигурация визуальной модели {'checkpoint', 'model_type', 'max_images', 'image_agg'}.
    :param audio_model_config: Конфигурация аудио модели {'checkpoint', 'model_type', 'max_audios', 'audio_agg', 'sr'}.
    :param freeze: Заморозить ли веса всех моделей.
    :param text_tokenizer_fn: Функция токенизации текста.
    :param special_tokens: Специальные токены.
    """
    name = "flexible_multi"
    
    def __init__(
        self,
        text_model_config: Optional[Dict[str, Any]] = None,
        image_model_config: Optional[Dict[str, Any]] = None,
        audio_model_config: Optional[Dict[str, Any]] = None,
        freeze: bool = True,
        text_tokenizer_fn: Optional[Callable] = None,
        special_tokens: Optional[Dict[str, str]] = None
    ):
        super().__init__()
        self.supported = set()
        self.out_dim_per_modality = {}
        self.text_tokenizer_fn = text_tokenizer_fn
        self.special_tokens = special_tokens or {"sep": " [SEP] "}
        
        # Инициализация текстовой модели
        self.text_model = None
        self.text_processor = None
        self.text_config = text_model_config or {}
        if text_model_config:
            self._init_text_model(text_model_config)
            self.supported.add("text")
        
        # Инициализация визуальной модели
        self.image_model = None
        self.image_processor = None
        self.image_config = image_model_config or {}
        if image_model_config:
            self._init_image_model(image_model_config)
            self.supported.add("image")
        
        # Инициализация аудио модели
        self.audio_model = None
        self.audio_processor = None
        self.audio_config = audio_model_config or {}
        if audio_model_config:
            self._init_audio_model(audio_model_config)
            self.supported.add("audio")
        
        if freeze:
            self.freeze_all()
    
    def _init_text_model(self, config: Dict[str, Any]):
        """
        Инициализирует текстовую модель согласно конфигурации.
        
        :param config: Словарь с 'checkpoint', 'model_type' и опциональными параметрами.
        """
        from transformers import AutoModel, AutoTokenizer, CLIPTextModel, CLIPTokenizer
        
        checkpoint = config['checkpoint']
        model_type = config.get('model_type', 'auto')
        
        if model_type == 'clip':
            self.text_model = CLIPTextModel.from_pretrained(checkpoint)
            self.text_processor = CLIPTokenizer.from_pretrained(checkpoint)
            # Для CLIP текстовой модели используем projection_dim
            dim = self.text_model.config.projection_dim
        elif model_type == 'bert' or model_type == 'auto':
            self.text_model = AutoModel.from_pretrained(checkpoint)
            self.text_processor = AutoTokenizer.from_pretrained(checkpoint)
            dim = self.text_model.config.hidden_size
        else:
            raise ValueError(f"Неизвестный model_type для текста: {model_type}")
        
        self.text_config['max_length'] = config.get('max_length', 512)
        self.text_config['dim'] = dim
        self.text_config['model_type'] = model_type
        self.out_dim_per_modality['text'] = dim
    
    def _init_image_model(self, config: Dict[str, Any]):
        """
        Инициализирует визуальную модель согласно конфигурации.
        
        :param config: Словарь с 'checkpoint', 'model_type' и опциональными параметрами.
        """
        from transformers import AutoModel, AutoImageProcessor, CLIPVisionModel, CLIPImageProcessor
        
        checkpoint = config['checkpoint']
        model_type = config.get('model_type', 'auto')
        
        if model_type == 'clip':
            self.image_model = CLIPVisionModel.from_pretrained(checkpoint)
            self.image_processor = CLIPImageProcessor.from_pretrained(checkpoint)
            # Для CLIPVisionModel используем hidden_size, так как get_image_features не проецирует
            dim = self.image_model.config.hidden_size
        elif model_type in ['dinov2', 'vit', 'auto']:
            self.image_model = AutoModel.from_pretrained(checkpoint)
            self.image_processor = AutoImageProcessor.from_pretrained(checkpoint)
            dim = self.image_model.config.hidden_size
        else:
            raise ValueError(f"Неизвестный model_type для изображений: {model_type}")
        
        self.image_config['max_images'] = config.get('max_images', 1)
        self.image_config['image_agg'] = config.get('image_agg', 'concat')
        self.image_config['dim'] = dim
        self.image_config['model_type'] = model_type
        
        if self.image_config['image_agg'] == 'concat':
            self.out_dim_per_modality['image'] = dim * self.image_config['max_images']
        else:
            self.out_dim_per_modality['image'] = dim
    
    def _init_audio_model(self, config: Dict[str, Any]):
        """
        Инициализирует аудио модель согласно конфигурации.
        
        :param config: Словарь с 'checkpoint', 'model_type' и опциональными параметрами.
        """
        from transformers import AutoModel, AutoProcessor, ClapAudioModel, ClapProcessor
        
        checkpoint = config['checkpoint']
        model_type = config.get('model_type', 'auto')
        
        if model_type == 'clap':
            from transformers import ClapModel
            self.audio_model = ClapModel.from_pretrained(checkpoint)
            self.audio_processor = ClapProcessor.from_pretrained(checkpoint)
            dim = getattr(self.audio_model.config, "projection_dim", 512)
            sr = getattr(self.audio_processor, "sampling_rate", None)
            if sr is None:
                fe = getattr(self.audio_processor, "feature_extractor", None)
                sr = getattr(fe, "sampling_rate", 48000) if fe is not None else 48000
        elif model_type in ['whisper', 'wav2vec2', 'auto']:
            self.audio_model = AutoModel.from_pretrained(checkpoint)
            self.audio_processor = AutoProcessor.from_pretrained(checkpoint)
            dim = self.audio_model.config.hidden_size
            sr = self.audio_processor.feature_extractor.sampling_rate
        else:
            raise ValueError(f"Неизвестный model_type для аудио: {model_type}")
        
        self.audio_config['sr'] = config.get('sr', sr)
        self.audio_config['max_audios'] = config.get('max_audios', 1)
        self.audio_config['audio_agg'] = config.get('audio_agg', 'concat')
        self.audio_config['dim'] = dim
        self.audio_config['model_type'] = model_type
        
        if self.audio_config['audio_agg'] == 'concat':
            self.out_dim_per_modality['audio'] = dim * self.audio_config['max_audios']
        else:
            self.out_dim_per_modality['audio'] = dim

    def collate(self, batch: List[Dict[str, Any]]) -> Dict[str, Any]:
        """
        Собирает батч для всех активных модальностей с корректной обработкой пропусков.
        """
        labels = torch.tensor(np.stack([b.get("labels", np.array([0.0])) for b in batch]), dtype=torch.float32)
        backend_inputs = {}
        batch_size = len(batch)
        
        # Обработка текста
        if self.text_model is not None:
            texts = []
            for b in batch:
                text = b.get("text", "")
                # Если текст пустой или None, используем пробел как заглушку
                texts.append(text if text else " ")
            
            text_inputs = self.text_processor(
                texts, padding=True, truncation=True,
                max_length=self.text_config.get('max_length', 512),
                return_tensors="pt"
            )
            backend_inputs["text_inputs"] = {k: v for k, v in text_inputs.items()}
        
        # Обработка изображений
        if self.image_model is not None:
            images_lists = [b.get("images", []) for b in batch]
            flat_images = []
            img_counts = []
            batch_indices = []  # Отслеживаем к какому сэмплу относится каждое изображение
            
            for idx, lst in enumerate(images_lists):
                lst = lst if isinstance(lst, list) else ([] if lst is None else [lst])
                # Фильтруем None и пустые значения
                lst = [img for img in lst if img is not None]
                img_counts.append(len(lst))
                for img in lst:
                    flat_images.append(to_pil(img))
                    batch_indices.append(idx)
            
            if len(flat_images) > 0:
                img_proc = self.image_processor(images=flat_images, return_tensors="pt")
                backend_inputs["image_inputs"] = {"pixel_values": img_proc["pixel_values"]}
            else:
                backend_inputs["image_inputs"] = {"pixel_values": None}
            
            backend_inputs["image_counts"] = torch.tensor(img_counts, dtype=torch.long)
            backend_inputs["image_batch_indices"] = batch_indices
        
        # Обработка аудио
        if self.audio_model is not None:
            audios_lists = [b.get("audios", []) for b in batch]
            flat_audios = []
            aud_counts = []
            audio_batch_indices = []
            
            for idx, lst in enumerate(audios_lists):
                lst = lst if isinstance(lst, list) else ([] if lst is None else [lst])
                # Фильтруем None и пустые значения
                lst = [a for a in lst if a is not None]
                aud_counts.append(len(lst))
                for a in lst:
                    if isinstance(a, str):
                        flat_audios.append(load_audio(a, self.audio_config['sr']))
                    elif isinstance(a, np.ndarray):
                        flat_audios.append(a.astype(np.float32))
                    audio_batch_indices.append(idx)
            
            if len(flat_audios) > 0:
                if self.audio_config.get('model_type') == 'clap':
                    aud_proc = self.audio_processor(
                        audios=flat_audios, 
                        sampling_rate=self.audio_config['sr'], 
                        padding=True, 
                        return_tensors="pt"
                    )
                    backend_inputs["audio_inputs"] = {"input_features": aud_proc["input_features"]}
                else:
                    aud_proc = self.audio_processor(
                        flat_audios, 
                        sampling_rate=self.audio_config['sr'],
                        padding=True,
                        return_tensors="pt"
                    )
                    backend_inputs["audio_inputs"] = {"input_values": aud_proc["input_values"]}
            else:
                backend_inputs["audio_inputs"] = {"input_features": None, "input_values": None}
            
            backend_inputs["audio_counts"] = torch.tensor(aud_counts, dtype=torch.long)
            backend_inputs["audio_batch_indices"] = audio_batch_indices
        
        backend_inputs["batch_size"] = batch_size
        return {"labels": labels, "backend_inputs": backend_inputs}

    def _aggregate_embeddings(
        self, 
        embs: Optional[torch.Tensor], 
        counts: List[int], 
        max_k: int, 
        dim: int, 
        agg_type: str,
        batch_size: int,
        device: torch.device
    ) -> torch.Tensor:
        """
        Безопасная агрегация эмбеддингов с правильной обработкой пустых сэмплов.
        """
        # Определяем актуальную размерность
        actual_dim = embs.size(1) if embs is not None and embs.numel() > 0 else dim
        
        # Создаем выходной тензор нужного размера
        if agg_type == 'concat':
            out_shape = (batch_size, actual_dim * max_k)
        else:  # mean
            out_shape = (batch_size, actual_dim)
        
        out = torch.zeros(out_shape, device=device, dtype=torch.float32)
        
        # Если нет эмбеддингов, возвращаем нули
        if embs is None or embs.numel() == 0:
            return out
        
        # Агрегируем эмбеддинги для каждого сэмпла
        offset = 0
        for i, count in enumerate(counts):
            if count > 0:
                sample_embs = embs[offset:offset + count]
                
                if agg_type == 'concat':
                    # Берем до max_k эмбеддингов
                    take = sample_embs[:max_k]
                    # Паддинг если нужно
                    if take.size(0) < max_k:
                        pad = torch.zeros((max_k - take.size(0), actual_dim), 
                                        device=device, dtype=embs.dtype)
                        take = torch.cat([take, pad], dim=0)
                    out[i] = take.reshape(-1)
                else:  # mean
                    out[i] = sample_embs.mean(dim=0)
                
                offset += count
        
        return F.normalize(out, dim=-1)
    
    def encode(self, backend_inputs: Dict[str, Any], device: torch.device) -> Dict[str, torch.Tensor]:
        """
        Кодирует все активные модальности через соответствующие модели.
        """
        results = {}
        
        # Определяем актуальный размер батча из labels или первого доступного тензора
        actual_batch_size = None
        
        # Пробуем определить размер батча из различных источников
        if "text_inputs" in backend_inputs:
            for v in backend_inputs["text_inputs"].values():
                if torch.is_tensor(v) and v.dim() > 0:
                    actual_batch_size = v.size(0)
                    break
        
        if actual_batch_size is None and "image_counts" in backend_inputs:
            actual_batch_size = len(backend_inputs["image_counts"])
        
        if actual_batch_size is None and "audio_counts" in backend_inputs:
            actual_batch_size = len(backend_inputs["audio_counts"])
        
        # Если все еще не определен, используем сохраненный
        if actual_batch_size is None:
            actual_batch_size = backend_inputs.get("batch_size", 1)
        
        # Кодирование текста
        if self.text_model is not None and "text_inputs" in backend_inputs:
            text_inputs = {k: v.to(device) for k, v in backend_inputs["text_inputs"].items()}
            
            # Обрезаем входы если нужно (для DataParallel)
            if text_inputs.get("input_ids") is not None:
                current_batch_size = text_inputs["input_ids"].size(0)
                if current_batch_size != actual_batch_size:
                    actual_batch_size = min(actual_batch_size, current_batch_size)
                    text_inputs = {k: v[:actual_batch_size] if torch.is_tensor(v) else v 
                                  for k, v in text_inputs.items()}
            
            if self.text_config.get('model_type') == 'clip':
                text_z = self.text_model.get_text_features(**text_inputs)
            elif hasattr(self.text_model, 'get_text_features'):
                text_z = self.text_model.get_text_features(**text_inputs)
            else:
                outputs = self.text_model(**text_inputs)
                if hasattr(outputs, 'pooler_output') and outputs.pooler_output is not None:
                    text_z = outputs.pooler_output
                else:
                    text_z = outputs.last_hidden_state.mean(dim=1)
            
            results["text"] = F.normalize(text_z, dim=-1)
            # Обновляем actual_batch_size на основе реального выхода
            actual_batch_size = text_z.size(0)
        
        # Кодирование изображений
        if self.image_model is not None and "image_inputs" in backend_inputs:
            pi = backend_inputs["image_inputs"]["pixel_values"]
            counts = backend_inputs["image_counts"]
            
            # Обрезаем counts до actual_batch_size
            if len(counts) > actual_batch_size:
                counts = counts[:actual_batch_size]
            counts = counts.tolist()
            
            # Проверяем, есть ли вообще изображения для обработки
            total_images_needed = sum(counts)
            
            if pi is not None and pi.numel() > 0 and total_images_needed > 0:
                pi = pi.to(device)
                
                # Обрезаем изображения согласно counts
                if pi.size(0) > total_images_needed:
                    pi = pi[:total_images_needed]
                
                if self.image_config.get('model_type') == 'clip':
                    outputs = self.image_model(pixel_values=pi)
                    img_flat = outputs.pooler_output
                elif hasattr(self.image_model, 'get_image_features'):
                    img_flat = self.image_model.get_image_features(pixel_values=pi)
                else:
                    outputs = self.image_model(pixel_values=pi)
                    if hasattr(outputs, 'pooler_output') and outputs.pooler_output is not None:
                        img_flat = outputs.pooler_output
                    else:
                        img_flat = outputs.last_hidden_state[:, 0]
                
                img_flat = F.normalize(img_flat, dim=-1)
                actual_img_dim = img_flat.size(1)
            else:
                img_flat = None
                actual_img_dim = self.image_config.get('dim', 768)
            
            # Используем безопасную агрегацию
            img_z = self._aggregate_embeddings(
                img_flat, counts,
                self.image_config['max_images'],
                actual_img_dim,
                self.image_config['image_agg'],
                len(counts),  # Используем длину counts как размер батча
                device
            )
            
            # Обновляем размерность если изменилась
            if actual_img_dim != self.image_config.get('dim'):
                self.image_config['dim'] = actual_img_dim
                if self.image_config['image_agg'] == 'concat':
                    self.out_dim_per_modality['image'] = actual_img_dim * self.image_config['max_images']
                else:
                    self.out_dim_per_modality['image'] = actual_img_dim
            
            results["image"] = img_z
        
        # Кодирование аудио
        if self.audio_model is not None and "audio_inputs" in backend_inputs:
            counts = backend_inputs["audio_counts"]
            
            # Обрезаем counts до actual_batch_size
            if len(counts) > actual_batch_size:
                counts = counts[:actual_batch_size]
            counts = counts.tolist()
            
            # Проверяем, есть ли вообще аудио для обработки
            total_audios_needed = sum(counts)
            
            # Получаем эмбеддинги аудио
            aud_flat = None
            actual_aud_dim = self.audio_config.get('dim', 768)
            
            if total_audios_needed > 0:  # Обрабатываем только если есть аудио
                if self.audio_config.get('model_type') == 'clap':
                    af = backend_inputs["audio_inputs"]["input_features"]
                    if af is not None and af.numel() > 0:
                        af = af.to(device)
                        
                        # Обрезаем аудио согласно counts
                        if af.size(0) > total_audios_needed:
                            af = af[:total_audios_needed]
                        
                        # Проверяем, что тензор не пустой после обрезки
                        if af.numel() > 0:
                            aud_flat = self.audio_model.get_audio_features(input_features=af)
                            aud_flat = F.normalize(aud_flat, dim=-1)
                            actual_aud_dim = aud_flat.size(1)
                else:
                    av = backend_inputs["audio_inputs"]["input_values"]
                    if av is not None and av.numel() > 0:
                        av = av.to(device)
                        
                        # Обрезаем аудио согласно counts
                        if av.size(0) > total_audios_needed:
                            av = av[:total_audios_needed]
                        
                        # Проверяем, что тензор не пустой после обрезки
                        if av.numel() > 0:
                            outputs = self.audio_model(input_values=av)
                            if hasattr(outputs, 'pooler_output') and outputs.pooler_output is not None:
                                aud_flat = outputs.pooler_output
                            else:
                                aud_flat = outputs.last_hidden_state.mean(dim=1)
                            aud_flat = F.normalize(aud_flat, dim=-1)
                            actual_aud_dim = aud_flat.size(1)
            
            # Используем безопасную агрегацию (она обработает None для aud_flat)
            aud_z = self._aggregate_embeddings(
                aud_flat, counts,
                self.audio_config['max_audios'],
                actual_aud_dim,
                self.audio_config['audio_agg'],
                len(counts),  # Используем длину counts как размер батча
                device
            )
            
            # Обновляем размерность если изменилась
            if aud_flat is not None and actual_aud_dim != self.audio_config.get('dim'):
                self.audio_config['dim'] = actual_aud_dim
                if self.audio_config['audio_agg'] == 'concat':
                    self.out_dim_per_modality['audio'] = actual_aud_dim * self.audio_config['max_audios']
                else:
                    self.out_dim_per_modality['audio'] = actual_aud_dim
            
            results["audio"] = aud_z
        
        # Убеждаемся, что все результаты имеют одинаковый размер батча
        if results:
            min_batch_size = min(v.size(0) for v in results.values())
            if any(v.size(0) != min_batch_size for v in results.values()):
                results = {k: v[:min_batch_size] for k, v in results.items()}
        
        return results


class SingleBackboneRegressor(nn.Module):
    """
    Регрессор поверх одного мультимодального бэкенда: encode -> fuse -> MLP голова.

    :param backend: Экземпляр бэкенда (CLIP/CLAP/FlexibleMultiBackend).
    :param modalities: Активные модальности (учёт порядка важен при concat): подмножество ['image','text','audio'].
    :param num_targets: Количество целевых переменных для регрессии.
    :param fusion: 'concat' (объединение признаков) или 'mean' (среднее по модальностям).
    :param hidden: Размер скрытого слоя головы.
    :param dropout: Дропаут в голове.
    """
    def __init__(
        self,
        backend: BaseBackend,
        modalities: List[str],
        num_targets: int,
        fusion: str = "concat",
        hidden: int = 512,
        dropout: float = 0.1
    ):
        super().__init__()
        self.backend = backend
        self.modalities = modalities
        self.fusion = fusion
        self.num_targets = num_targets

        order = [m for m in ["image", "text", "audio"] if m in self.modalities]
        if fusion == "concat":
            in_dim = sum(self.backend.get_out_dim(m) for m in order)
        elif fusion == "mean":
            dims = [self.backend.get_out_dim(m) for m in order]
            if len(set(dims)) != 1:
                raise ValueError(f'Для fusion="mean" размеры модальностей должны совпадать, а у нас: {dict(zip(order, dims))}')
            in_dim = dims[0]
        else:
            raise ValueError('fusion должен быть "concat" или "mean"')

        self.head = nn.Sequential(
            nn.LayerNorm(in_dim),
            nn.Linear(in_dim, hidden),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden, num_targets)
        )

    def _infer_device_from_inputs(self, obj) -> torch.device:
        """
        Находит девайс по первому тензору во входах; иначе выбирает доступный cuda/cpu.

        :param obj: Любая структура с тензорами.
        :return: torch.device.
        """
        if isinstance(obj, torch.Tensor):
            return obj.device
        if isinstance(obj, dict):
            for v in obj.values():
                d = self._infer_device_from_inputs(v)
                if d is not None:
                    return d
        return torch.device("cuda" if torch.cuda.is_available() else "cpu")

    def _fuse(self, z: Dict[str, torch.Tensor]) -> torch.Tensor:
        """
        Объединяет эмбеддинги модальностей согласно self.fusion.

        :param z: Словарь эмбеддингов по модальностям.
        :return: Fused тензор [B, *].
        """
        order = [m for m in ["image", "text", "audio"] if m in self.modalities]
        feats = []
        batch_size = None
        
        for m in order:
            if m in z:
                t = z[m]
                if t.dim() == 3:
                    t = t.mean(dim=1)
                elif t.dim() > 3:
                    t = t.view(t.size(0), -1)
                feats.append(t)
                if batch_size is None:
                    batch_size = t.size(0)
        
        # Убеждаемся, что все тензоры имеют одинаковый размер батча
        if batch_size is not None:
            feats = [f[:batch_size] for f in feats]
        
        if self.fusion == "concat":
            return torch.cat(feats, dim=-1)
        elif self.fusion == "mean":
            sizes = [f.size(-1) for f in feats]
            if len(set(sizes)) != 1:
                raise ValueError(f'Для fusion="mean" размеры модальностей должны совпадать, а у нас: {sizes}')
            return torch.stack(feats, dim=0).mean(dim=0)

    def forward(self, backend_inputs: Dict[str, Any], labels: Optional[torch.Tensor] = None):
        """
        Прямой проход модели.

        :param backend_inputs: Входы для бэкенда (из его collate).
        :param labels: Игнорируется (loss считает Trainer).
        :return: SequenceClassifierOutput с logits [B, num_targets].
        """
        device = self._infer_device_from_inputs(backend_inputs)
        
        # Кодируем модальности
        z = self.backend.encode(backend_inputs, device=device)
        
        # Проверяем, что получили эмбеддинги
        if not z:
            raise ValueError("Backend не вернул эмбеддинги")
        
        # Объединяем эмбеддинги модальностей
        fused = self._fuse(z)
        
        # Пропускаем через регрессионную голову
        logits = self.head(fused)
        
        return SequenceClassifierOutput(logits=logits)

    @torch.no_grad()
    def get_embeddings(self, backend_inputs: Dict[str, Any], return_per_modality: bool = False):
        """
        Извлекает fused эмбеддинги (и опционально по модальностям).

        :param backend_inputs: Входы для бэкенда.
        :param return_per_modality: Вернуть также словарь {'text','image','audio'}.
        :return: fused [B, *] или (fused, per_modality).
        """
        device = self._infer_device_from_inputs(backend_inputs)
        z = self.backend.encode(backend_inputs, device=device)
        fused = self._fuse(z)
        if return_per_modality:
            return fused, z
        return fused


class RegressionTrainer(Trainer):
    """
    Trainer с MSELoss для регрессии множественных целевых переменных.
    Поддерживает опциональные веса для разных целевых переменных.

    :param num_targets: Количество целевых переменных.
    :param target_weights: Опциональные веса для каждой целевой переменной.
    :param loss_type: Тип функции потерь ('mse', 'mae', 'huber').
    """
    def __init__(self, *args, num_targets=None, target_weights=None, loss_type='mse', **kwargs):
        super().__init__(*args, **kwargs)
        self.num_targets = num_targets
        self.loss_type = loss_type.lower()
        
        if target_weights is not None:
            self.target_weights = torch.as_tensor(target_weights, dtype=torch.float32)
        else:
            self.target_weights = None

    def compute_loss(self, model, inputs, return_outputs: bool = False, num_items_in_batch: Optional[int] = None):
        """
        Считает регрессионную функцию потерь (MSE/MAE/Huber).
        Корректно обрабатывает DataParallel и защищается от NaN.

        :param model: Модель.
        :param inputs: Батч: {'labels': FloatTensor[B, num_targets], 'backend_inputs': {...}}.
        :param return_outputs: Возвращать ли outputs вместе с loss.
        :param num_items_in_batch: Совместимость с Trainer API (не используется).
        :return: loss (и outputs, если return_outputs=True).
        """
        labels = inputs.pop("labels")
        
        # Проверяем, используется ли DataParallel
        is_parallel = isinstance(model, (nn.DataParallel, nn.parallel.DistributedDataParallel))
        
        # Вызываем forward модели
        outputs = model(**inputs)
        predictions = outputs.logits
        
        # Переносим labels на устройство predictions
        labels = labels.to(predictions.device)
        
        # Обработка несоответствия размеров при DataParallel
        if predictions.size(0) != labels.size(0):
            # Если predictions меньше labels (может быть при разделении батча между GPU)
            if predictions.size(0) < labels.size(0):
                # Обрезаем labels до размера predictions
                labels = labels[:predictions.size(0)]
            # Если predictions больше labels (DataParallel может дублировать)
            elif is_parallel:
                # Для DataParallel: повторяем labels для каждой реплики
                num_replicas = predictions.size(0) // labels.size(0)
                if predictions.size(0) == labels.size(0) * num_replicas:
                    labels = labels.repeat_interleave(num_replicas, dim=0)
                else:
                    # Если размеры не кратны, берем первые predictions.size(0) элементов
                    labels = labels.repeat(num_replicas + 1, dim=0)[:predictions.size(0)]
            else:
                # В других случаях просто обрезаем до минимального размера
                min_size = min(predictions.size(0), labels.size(0))
                predictions = predictions[:min_size]
                labels = labels[:min_size]
        
        # Проверка на NaN и Inf в predictions
        if torch.isnan(predictions).any() or torch.isinf(predictions).any():
            # Заменяем NaN и Inf на нули
            predictions = torch.nan_to_num(predictions, nan=0.0, posinf=1e4, neginf=-1e4)
        
        # Вычисляем loss
        try:
            if self.loss_type == 'mse':
                loss = F.mse_loss(predictions, labels, reduction='none')
            elif self.loss_type == 'mae':
                loss = F.l1_loss(predictions, labels, reduction='none')
            elif self.loss_type == 'huber':
                loss = F.huber_loss(predictions, labels, reduction='none', delta=1.0)
            else:
                raise ValueError(f"Неизвестный тип функции потерь: {self.loss_type}")
            
            # Применяем веса если есть
            if self.target_weights is not None:
                weights = self.target_weights.to(loss.device)
                # Расширяем веса до размера батча
                if weights.dim() == 1 and loss.dim() == 2:
                    weights = weights.unsqueeze(0).expand_as(loss)
                loss = loss * weights
            
            # Усредняем по всем измерениям
            loss = loss.mean()
            
            # Проверка на NaN в loss
            if torch.isnan(loss) or torch.isinf(loss):
                # Если loss NaN, возвращаем малое значение
                loss = torch.tensor(0.01, device=predictions.device, requires_grad=True)
                
        except Exception as e:
            # В случае любой ошибки возвращаем малое значение loss
            print(f"Warning: Error computing loss: {e}")
            loss = torch.tensor(0.01, device=predictions.device, requires_grad=True)
        
        return (loss, outputs) if return_outputs else loss


class PbarConsoleLogger(TrainerCallback):
    def __init__(self, pbar):
        self.pbar = pbar
        self.last_logs = {}
        self.last_train_loss = None
        self.printed_eval_steps = set()

    def _step(self, state) -> int:
        return int(state.global_step or 0)

    def _fmt_postfix(self):
        parts = []
        if 'loss' in self.last_logs:
            parts.append(f"loss {self.last_logs['loss']:.4f}")
        if 'eval_loss' in self.last_logs:
            parts.append(f"val {self.last_logs['eval_loss']:.4f}")
        for k, v in self.last_logs.items():
            if k.startswith('eval_') and k not in (
                'eval_loss', 'eval_runtime', 'eval_samples_per_second', 'eval_steps_per_second', 'epoch'
            ):
                parts.append(f"{k.replace('eval_', '')} {v:.4f}")
        return " | ".join(parts)

    def on_step_end(self, args, state, control, **kwargs):
        n = min(self._step(state), self.pbar.total)
        if n > self.pbar.n:
            self.pbar.update(n - self.pbar.n)
        if self.last_logs:
            self.pbar.set_postfix_str(self._fmt_postfix(), refresh=False)
        self.pbar.refresh()

    def on_log(self, args, state, control, logs=None, **kwargs):
        if not logs:
            return

        for k, v in logs.items():
            if isinstance(v, (int, float)):
                self.last_logs[k] = float(v)
        if 'loss' in logs and isinstance(logs['loss'], (int, float)):
            self.last_train_loss = float(logs['loss'])

        self.pbar.set_postfix_str(self._fmt_postfix(), refresh=False)
        self.pbar.refresh()

        if any(k.startswith('eval_') for k in logs.keys()):
            step = self._step(state)
            if step in self.printed_eval_steps:
                return
            self.printed_eval_steps.add(step)

            train_loss_str = f"{self.last_train_loss:.10f}" if self.last_train_loss is not None else "n/a"
            val_loss = logs.get('eval_loss', None)
            val_loss_str = f"{float(val_loss):.10g}" if isinstance(val_loss, (int, float)) else "n/a"

            exclude = {'eval_loss', 'eval_runtime', 'eval_samples_per_second', 'eval_steps_per_second', 'epoch'}
            extra_parts = []
            for k, v in logs.items():
                if k.startswith('eval_') and k not in exclude:
                    metric_name = k.replace('eval_', '')
                    extra_parts.append(f"val {metric_name}: {float(v):.10f}")

            line = f"step: {step}, train loss: {train_loss_str}, val loss: {val_loss_str}"
            if extra_parts:
                line += ", " + ", ".join(extra_parts)
            tqdm.write(line)

    def on_train_end(self, args, state, control, **kwargs):
        n = min(self._step(state), self.pbar.total)
        if n > self.pbar.n:
            self.pbar.update(n - self.pbar.n)
        self.pbar.refresh()


class SingleModelMultiComboRegression:
    """
    Пайплайн: одна мультимодальная модель (бэкенд) + регрессионная голова + HuggingFace Trainer.
    Поддерживает регрессию множественных целевых переменных.

    Поддерживаемые комбинации модальностей:
      - ['text','image']         -> ClipBackend или FlexibleMultiBackend
      - ['text','audio']         -> ClapBackend или FlexibleMultiBackend
      - ['image','audio']        -> FlexibleMultiBackend
      - ['text','image','audio'] -> FlexibleMultiBackend

    Возможности:
      - Мульти-изображения/аудио per-сэмпл (concat/mean агрегация).
      - Обучение на больших данных: чанковая подстановка train_dataset, стабильный прогресс‑бар и логи.
      - Множественные целевые переменные с опциональными весами.
      - Гибкая архитектура: возможность использовать разные модели для разных модальностей.
      - Кастомная токенизация текста через переданную функцию.
      - Различные функции потерь (MSE, MAE, Huber).

    Необходимые импорты:
    !pip install -q wav2clip torchaudio evaluate pillow
    import gc
    import math
    from typing import List, Dict, Any, Optional, Union, Tuple, Callable
    import numpy as np
    import pandas as pd
    from PIL import Image
    import torch
    import torch.nn as nn
    import torch.nn.functional as F
    from torch.utils.data import Dataset, DataLoader
    import evaluate
    from transformers import Trainer, TrainingArguments, TrainerCallback, PrinterCallback
    from transformers.modeling_outputs import SequenceClassifierOutput
    from tqdm.auto import tqdm
    """
    def __init__(
        self,
        modalities: List[str],
        num_targets: int,
        target_column_names: List[str],
        text_columns: Optional[List[str]] = None,
        image_columns: Optional[List[str]] = None,
        audio_columns: Optional[List[str]] = None,
        text_tokenizer_fn: Optional[Callable] = None,
        special_tokens: Optional[Dict[str, str]] = None,
        backend: str = "auto",
        clip_checkpoint: str = "openai/clip-vit-base-patch32",
        clap_checkpoint: str = "laion/clap-htsat-unfused",
        text_model_config: Optional[Dict[str, Any]] = None,
        image_model_config: Optional[Dict[str, Any]] = None,
        audio_model_config: Optional[Dict[str, Any]] = None,
        fusion: str = "concat",
        freeze_backbone: bool = True,
        clip_max_length: int = 77,
        max_images_per_sample: int = 1,
        max_audios_per_sample: int = 1,
        target_normalizer: Optional[Callable] = None,
        target_denormalizer: Optional[Callable] = None
    ):
        """
        :param modalities: Список модальностей ('text','image','audio') в любом порядке.
        :param num_targets: Количество целевых переменных.
        :param target_column_names: Список имён столбцов целевых переменных в DataFrame.
        :param text_columns: Имена текстовых колонок.
        :param image_columns: Имена колонок изображений (значения — пути или списки путей/объектов).
        :param audio_columns: Имена колонок аудио (значения — пути/массивы или списки).
        :param text_tokenizer_fn: Функция токенизации текста. Принимает dict колонок и special_tokens.
        :param special_tokens: Специальные токены для токенизатора.
        :param backend: 'auto' | 'clip' | 'clap' | 'flexible'.
        :param clip_checkpoint: Чекпоинт CLIP (для backend='clip').
        :param clap_checkpoint: Чекпоинт CLAP (для backend='clap').
        :param text_model_config: Конфиг текстовой модели для FlexibleMultiBackend.
        :param image_model_config: Конфиг визуальной модели для FlexibleMultiBackend.
        :param audio_model_config: Конфиг аудио модели для FlexibleMultiBackend.
        :param fusion: 'concat' или 'mean' — тип фьюжна эмбеддингов.
        :param freeze_backbone: Заморозить веса бэкенда (linear probing).
        :param clip_max_length: Максимальная длина токенов в CLIP.
        :param max_images_per_sample: Максимум картинок при concat-агрегации.
        :param max_audios_per_sample: Максимум аудио при concat-агрегации.
        :param target_normalizer: Опциональная функция нормализации целевых значений.
        :param target_denormalizer: Опциональная функция денормализации предсказаний.
        """
        self.modalities = sorted(list(set(modalities)))
        self.num_targets = num_targets
        self.target_column_names = target_column_names
        self.text_columns = text_columns or []
        self.image_columns = image_columns or []
        self.audio_columns = audio_columns or []
        self.text_tokenizer_fn = text_tokenizer_fn
        self.special_tokens = special_tokens or {"sep": " [SEP] "}
        self.backend_name = backend
        self.clip_checkpoint = clip_checkpoint
        self.clap_checkpoint = clap_checkpoint
        self.text_model_config = text_model_config
        self.image_model_config = image_model_config
        self.audio_model_config = audio_model_config
        self.fusion = fusion
        self.freeze_backbone = freeze_backbone
        self.clip_max_length = clip_max_length
        self.max_images_per_sample = int(max_images_per_sample)
        self.max_audios_per_sample = int(max_audios_per_sample)
        self.target_normalizer = target_normalizer
        self.target_denormalizer = target_denormalizer

        # Проверка соответствия
        if len(self.target_column_names) != self.num_targets:
            raise ValueError(f"Количество target_column_names ({len(self.target_column_names)}) "
                           f"не соответствует num_targets ({self.num_targets})")

        self.backend: Optional[BaseBackend] = None
        self.model: Optional[SingleBackboneRegressor] = None
        self.trainer: Optional[Trainer] = None
        self.compute_metrics = None
        self.progress_callback: Optional[PbarConsoleLogger] = None

        self._build_backend()

    def _build_backend(self):
        """
        Инициализирует бэкенд согласно backend='auto'|'clip'|'clap'|'flexible' и проверяет совместимость модальностей.

        :raises ValueError: При неподдерживаемой комбинации модальностей.
        """
        mods = set(self.modalities)
        name = self.backend_name
        if name == "auto":
            if mods == {"text", "image"}:
                name = "clip"
            elif mods == {"text", "audio"}:
                name = "clap"
            else:
                name = "flexible"

        if name == "clip":
            self.backend = ClipBackend(
                checkpoint=self.clip_checkpoint,
                max_length=self.clip_max_length,
                freeze=self.freeze_backbone,
                max_images=self.max_images_per_sample,
                image_agg="concat",
                text_tokenizer_fn=self.text_tokenizer_fn,
                special_tokens=self.special_tokens
            )
        elif name == "clap":
            self.backend = ClapBackend(
                checkpoint=self.clap_checkpoint,
                freeze=self.freeze_backbone,
                max_audios=self.max_audios_per_sample,
                audio_agg="concat",
                text_tokenizer_fn=self.text_tokenizer_fn,
                special_tokens=self.special_tokens
            )
        elif name == "flexible":
            # Автоматическая конфигурация если не заданы модели
            if "text" in mods and self.text_model_config is None:
                self.text_model_config = {
                    'checkpoint': 'bert-base-uncased',
                    'model_type': 'bert',
                    'max_length': 512
                }
            if "image" in mods and self.image_model_config is None:
                self.image_model_config = {
                    'checkpoint': 'google/vit-base-patch16-224',
                    'model_type': 'vit',
                    'max_images': self.max_images_per_sample,
                    'image_agg': 'concat'
                }
            if "audio" in mods and self.audio_model_config is None:
                self.audio_model_config = {
                    'checkpoint': self.clap_checkpoint,
                    'model_type': 'clap',
                    'max_audios': self.max_audios_per_sample,
                    'audio_agg': 'concat',
                    'sr': 48000
                }
            
            self.backend = FlexibleMultiBackend(
                text_model_config=self.text_model_config if "text" in mods else None,
                image_model_config=self.image_model_config if "image" in mods else None,
                audio_model_config=self.audio_model_config if "audio" in mods else None,
                freeze=self.freeze_backbone,
                text_tokenizer_fn=self.text_tokenizer_fn,
                special_tokens=self.special_tokens
            )
        else:
            raise ValueError(f"Неизвестный backend: {name}")

        if not set(self.modalities).issubset(self.backend.supported):
            raise ValueError(f"Бэкенд {self.backend.name} не поддерживает модальности {self.modalities}. "
                             f"Поддерживает: {self.backend.supported}")

    def _setup_metrics(self, metric_names: Union[str, List[str]]):
        """
        Создаёт функцию подсчёта метрик для Trainer.

        :param metric_names: 'mse', 'mae', 'r2' или список таких метрик.
        """
        if isinstance(metric_names, str):
            metric_names = [metric_names]
        
        metric_names = [m.lower() for m in metric_names]
        
        def compute(p):
            preds = p.predictions
            labels = p.label_ids
            results = {}
            
            for metric_name in metric_names:
                if metric_name == "mse":
                    mse = np.mean((preds - labels) ** 2)
                    results["mse"] = float(mse)
                elif metric_name == "mae":
                    mae = np.mean(np.abs(preds - labels))
                    results["mae"] = float(mae)
                elif metric_name == "r2":
                    # R² score для каждой целевой переменной, затем усредняем
                    from sklearn.metrics import r2_score
                    if preds.ndim == 1:
                        r2 = r2_score(labels, preds)
                    else:
                        r2_scores = []
                        for i in range(preds.shape[1]):
                            r2_scores.append(r2_score(labels[:, i], preds[:, i]))
                        r2 = np.mean(r2_scores)
                    results["r2"] = float(r2)
                else:
                    raise ValueError(f'Неизвестная метрика: {metric_name}')
            
            return results
        
        self.compute_metrics = compute

    def _split(self, df: pd.DataFrame, test_size: float = 0.2, seed: int = 42):
        """
        Перемешивает и делит DataFrame на train/eval.

        :param df: Полный датафрейм.
        :param test_size: Доля валидации (0..1).
        :param seed: Зерно.
        :return: (df_train, df_eval).
        """
        df = df.sample(frac=1.0, random_state=seed).reset_index(drop=True)
        n_eval = int(math.ceil(len(df) * test_size))
        return df.iloc[n_eval:].reset_index(drop=True), df.iloc[:n_eval].reset_index(drop=True)

    def _validate_data(self, df: pd.DataFrame):
        """
        Проверяет соответствие колонок выбранным модальностям и наличие целевых колонок.

        :param df: Источник данных.
        :raises ValueError: При отсутствии необходимых колонок.
        """
        # Проверка целевых колонок
        missing_targets = [c for c in self.target_column_names if c not in df.columns]
        if missing_targets:
            raise ValueError(f"В DataFrame отсутствуют целевые колонки: {missing_targets}")
        
        # Проверка модальностей
        if "text" in self.modalities:
            if not self.text_columns:
                raise ValueError("Вы выбрали модальность 'text', но text_columns пустой.")
            missing = [c for c in self.text_columns if c not in df.columns]
            if missing:
                raise ValueError(f"В DataFrame отсутствуют текстовые колонки: {missing}")

        if "image" in self.modalities:
            if not self.image_columns:
                raise ValueError("Вы выбрали модальность 'image', но image_columns пуст.")
            missing = [c for c in self.image_columns if c not in df.columns]
            if missing:
                raise ValueError(f"В DataFrame отсутствуют колонки изображений: {missing}")

        if "audio" in self.modalities:
            if not self.audio_columns:
                raise ValueError("Вы выбрали модальность 'audio', но audio_columns пуст.")
            missing = [c for c in self.audio_columns if c not in df.columns]
            if missing:
                raise ValueError(f"В DataFrame отсутствуют колонки аудио: {missing}")

    def fit(
        self,
        train_data: pd.DataFrame,
        epochs: int = 3,
        test_size: float = 0.2,
        per_device_train_batch_size: int = 16,
        gradient_accumulation_steps: int = 1,
        learning_rate: float = 2e-4,
        metric_names: Union[str, List[str]] = "mse",
        loss_type: str = "mse",
        target_weights: Optional[List[float]] = None,
        fp16: bool = True,
        logging_steps: int = 50,
        eval_steps: int = 200,
        output_dir: str = "./result",
        seed: int = 42,
        hidden: int = 512,
        dropout: float = 0.1,
        gradient_checkpointing: bool = False,
        fit_chunk_size: Optional[int] = None
    ):
        """
        Обучает регрессионную голову поверх выбранного бэкенда.
        Поддерживает обучение на больших данных за счёт чанков: train_dataset подставляется кусками.

        :param train_data: Полный датафрейм с данными и таргетами.
        :param epochs: Количество эпох.
        :param test_size: Доля валидации.
        :param per_device_train_batch_size: Размер батча на устройство.
        :param gradient_accumulation_steps: Шаги аккумуляции градиента.
        :param learning_rate: Learning rate для оптимизатора.
        :param metric_names: 'mse', 'mae', 'r2' или список метрик для валидации.
        :param loss_type: 'mse', 'mae' или 'huber' — функция потерь для обучения.
        :param target_weights: Опциональные веса для каждой целевой переменной.
        :param fp16: Использовать fp16 при наличии CUDA (если доступен bf16 — он будет использован вместо fp16).
        :param logging_steps: Частота логирования шагов.
        :param eval_steps: Шаги между валидациями/сохранениями.
        :param output_dir: Каталог для артефактов.
        :param seed: Зерно.
        :param hidden: Размер скрытого слоя головы.
        :param dropout: Дропаут в голове.
        :param gradient_checkpointing: Делать ли чекпоинты во время обучения для экономии VRAM.
        :param fit_chunk_size: Размер чанка обучающей выборки. Если None — весь train как один чанк.
        :return: self.
        """
        self._validate_data(train_data)
        set_seed(seed)

        df_train, df_eval = self._split(train_data, test_size=test_size, seed=seed)

        # Датасет валидации держим целиком (обычно небольшой).
        ds_eval = MultiComboRegressionDataset(
            df_eval, self.target_column_names,
            self.text_columns, self.image_columns, self.audio_columns,
            self.text_tokenizer_fn, self.special_tokens,
            self.target_normalizer
        )

        # Модель и метрики
        self.model = SingleBackboneRegressor(
            backend=self.backend,
            modalities=self.modalities,
            num_targets=self.num_targets,
            fusion=self.fusion,
            hidden=hidden,
            dropout=dropout
        )
        self._setup_metrics(metric_names)

        # Определяем метрику для выбора лучшей модели
        if isinstance(metric_names, str):
            best_metric = f"eval_{metric_names}"
        else:
            # Используем первую метрику из списка
            best_metric = f"eval_{metric_names[0]}"

        # Настройки точности
        bf16_ok = bool(torch.cuda.is_available() and torch.cuda.get_device_capability()[0] >= 8)

        args = TrainingArguments(
            output_dir=output_dir,
            num_train_epochs=epochs,
            per_device_train_batch_size=per_device_train_batch_size,
            per_device_eval_batch_size=max(4, per_device_train_batch_size // 2),
            gradient_accumulation_steps=gradient_accumulation_steps,
            eval_accumulation_steps=max(1, gradient_accumulation_steps * 2),
            learning_rate=learning_rate,
            warmup_ratio=0.1,
            lr_scheduler_type="cosine",
            weight_decay=0.01,
            eval_strategy="steps",
            eval_steps=eval_steps,
            save_strategy="steps",
            save_steps=eval_steps,
            load_best_model_at_end=True,
            metric_for_best_model=best_metric,
            greater_is_better=best_metric == "eval_r2",  # Для R² больше — лучше, для MSE/MAE меньше
            save_total_limit=1,
            logging_strategy="steps",
            logging_steps=logging_steps,
            report_to="none",
            fp16=bool(fp16 and torch.cuda.is_available() and not bf16_ok),
            bf16=bool(bf16_ok and not fp16),
            dataloader_num_workers=os.cpu_count(),
            seed=seed,
            remove_unused_columns=False,
            gradient_checkpointing=gradient_checkpointing,
            dataloader_pin_memory=True,
            ddp_find_unused_parameters=False,
            disable_tqdm=True
        )

        def data_collator(batch_list: List[Dict[str, Any]]) -> Dict[str, Any]:
            """
            Адаптер collate для Trainer: делегирует бэкенду сборку батча.

            :param batch_list: Список элементов Dataset.
            :return: Батч для model.forward(): {'labels': FloatTensor, 'backend_inputs': {...}}.
            """
            return self.backend.collate(batch_list)

        # Вспомогательные функции для чанков
        def steps_for_size(sz: int, bsz: int, accum: int) -> int:
            """
            Оценивает число шагов оптимизации на чанке размера sz.

            :param sz: Количество примеров в чанке.
            :param bsz: Размер батча.
            :param accum: Шаги аккумуляции.
            :return: Число оптимизационных шагов.
            """
            return max(0, math.ceil(math.ceil(sz / max(1, bsz)) / max(1, accum)))

        def chunk_slices(index_array: np.ndarray, chunk_size: int):
            """
            Генератор срезов индексов по chunk_size.

            :param index_array: Индексы обучающей выборки.
            :param chunk_size: Размер чанка.
            :yield: Срез индексов.
            """
            for i in range(0, len(index_array), chunk_size):
                yield index_array[i:i + chunk_size]

        # Индексы train
        n_train = len(df_train)
        rng = np.random.default_rng(seed)
        train_idx = np.arange(n_train)

        # Чанк по умолчанию — весь train
        chunk_size = fit_chunk_size if (fit_chunk_size and fit_chunk_size > 0) else len(train_idx)

        # Предварительный рассчёт общего числа шагов (для прогресс‑бара и планировщика)
        total_steps = 0
        for _ in range(epochs):
            rng.shuffle(train_idx)
            for slc in chunk_slices(train_idx, chunk_size):
                total_steps += steps_for_size(len(slc), per_device_train_batch_size, gradient_accumulation_steps)

        # Инициализация Trainer с «пустым» train датасетом (минимальный чанк), чтобы не держать весь train
        dummy_idx = np.arange(min(len(df_train), 1))
        ds_train_init = MultiComboRegressionDataset(
            df_train.iloc[dummy_idx], self.target_column_names,
            self.text_columns, self.image_columns, self.audio_columns,
            self.text_tokenizer_fn, self.special_tokens,
            self.target_normalizer
        ) if len(dummy_idx) > 0 else ds_eval

        self.trainer = RegressionTrainer(
            model=self.model,
            args=args,
            train_dataset=ds_train_init,
            eval_dataset=ds_eval,
            data_collator=data_collator,
            compute_metrics=self.compute_metrics,
            num_targets=self.num_targets,
            target_weights=target_weights,
            loss_type=loss_type
        )
        self.trainer.remove_callback(PrinterCallback)

        # Планировщик на рассчитанное количество шагов
        if total_steps > 0:
            self.trainer.create_optimizer_and_scheduler(num_training_steps=total_steps)

        # Внешний прогресс‑бар + консольный лог
        pbar = tqdm(total=total_steps, desc="Training Progress", unit="step")
        cb = PbarConsoleLogger(pbar)
        self.trainer.add_callback(cb)
        self.progress_callback = cb

        # Основной цикл обучения по эпохам и чанкам
        steps_done = 0
        for ep in range(epochs):
            rng = np.random.default_rng(seed + ep)
            shuffled = np.arange(n_train)
            rng.shuffle(shuffled)

            for slc in chunk_slices(shuffled, chunk_size):
                # Подставляем чанк
                chunk_df = df_train.iloc[slc]
                ds_chunk = MultiComboRegressionDataset(
                    chunk_df, self.target_column_names,
                    self.text_columns, self.image_columns, self.audio_columns,
                    self.text_tokenizer_fn, self.special_tokens,
                    self.target_normalizer
                )
                self.trainer.train_dataset = ds_chunk

                # Считаем шаги на чанке и настраиваем max_steps Trainer
                chunk_steps = steps_for_size(len(slc), per_device_train_batch_size, gradient_accumulation_steps)
                if chunk_steps == 0:
                    del ds_chunk, chunk_df
                    continue

                self.trainer.args.max_steps = steps_done + chunk_steps
                self.trainer.train()
                steps_done += chunk_steps

                # Очистка памяти между чанками
                del ds_chunk, chunk_df
                gc.collect()
                if torch.cuda.is_available():
                    for i in range(torch.cuda.device_count()):
                        with torch.cuda.device(i):
                            torch.cuda.empty_cache()
                            torch.cuda.ipc_collect()  # Очистка IPC памяти
                    
                    # Синхронизация всех GPU (важно при DataParallel)
                    if torch.cuda.device_count() > 1:
                        for i in range(torch.cuda.device_count()):
                            torch.cuda.synchronize(i)
                    
                    # Дополнительная очистка (если используется)
                    if hasattr(torch.cuda, 'reset_peak_memory_stats'):
                        for i in range(torch.cuda.device_count()):
                            torch.cuda.reset_peak_memory_stats(i)

        pbar.close()
        return self

    def predict(self, df: pd.DataFrame, return_denormalized: bool = True) -> np.ndarray:
        """
        Делает предсказания регрессионных значений на новых данных.

        :param df: Датафрейм с теми же колонками модальностей, что и при обучении.
        :param return_denormalized: Если True и есть denormalizer — применить денормализацию.
        :return: np.ndarray предсказанных значений [N, num_targets].
        :raises RuntimeError: Если модель ещё не обучена.
        """
        if self.trainer is None:
            raise RuntimeError("Модель не обучена. Вызовите .fit().")
        
        df_c = df.copy()
        # Добавляем фиктивные целевые колонки если их нет
        for col in self.target_column_names:
            if col not in df_c.columns:
                df_c[col] = 0.0
        
        ds = MultiComboRegressionDataset(
            df_c, self.target_column_names,
            self.text_columns, self.image_columns, self.audio_columns,
            self.text_tokenizer_fn, self.special_tokens,
            self.target_normalizer
        )
        
        preds = self.trainer.predict(test_dataset=ds)
        predictions = preds.predictions
        
        # Применяем денормализацию если нужно
        if return_denormalized and self.target_denormalizer is not None:
            predictions = np.array([self.target_denormalizer(p) for p in predictions])
        
        return predictions

    def get_embeddings(self, df: pd.DataFrame, batch_size: int = 32, return_per_modality: bool = False):
        """
        Извлекает fused эмбеддинги (и опционально по модальностям) для новых данных.

        :param df: Датафрейм с нужными колонками модальностей.
        :param batch_size: Размер батча при инференсе.
        :param return_per_modality: Вернуть также словарь эмбеддингов {'text','image','audio'}.
        :return: np.ndarray fused [N, D_fused] или (fused, per_modality_dict).
        :raises RuntimeError: Если модель ещё не обучена.
        """
        if self.trainer is None or self.model is None:
            raise RuntimeError("Модель не обучена. Вызовите .fit().")

        # Определяем девайс модели
        try:
            device = next(self.trainer.model.parameters()).device
        except StopIteration:
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        self.model.to(device).eval()

        df_c = df.copy()
        # Добавляем фиктивные целевые колонки если их нет
        for col in self.target_column_names:
            if col not in df_c.columns:
                df_c[col] = 0.0

        ds = MultiComboRegressionDataset(
            df_c, self.target_column_names,
            self.text_columns, self.image_columns, self.audio_columns,
            self.text_tokenizer_fn, self.special_tokens,
            self.target_normalizer
        )

        def collate(batch_list: List[Dict[str, Any]]) -> Dict[str, Any]:
            """
            Collate для DataLoader при извлечении эмбеддингов.

            :param batch_list: Список элементов.
            :return: Батч 'backend_inputs' для модели.
            """
            return self.backend.collate(batch_list)

        def move_to_device(obj, device: torch.device):
            """
            Рекурсивно переносит тензоры на device.

            :param obj: Тензор/словарь/список/кортеж/прочее.
            :param device: torch.device.
            :return: Объект с перенесёнными тензорами.
            """
            if torch.is_tensor(obj):
                return obj.to(device)
            if isinstance(obj, dict):
                return {k: move_to_device(v, device) for k, v in obj.items()}
            if isinstance(obj, (list, tuple)):
                t = [move_to_device(v, device) for v in obj]
                return type(obj)(t) if not isinstance(obj, list) else t
            return obj

        loader = DataLoader(ds, batch_size=batch_size, shuffle=False, collate_fn=collate)
        fused_list = []
        per_mod_lists = {m: [] for m in self.modalities} if return_per_modality else None

        with torch.no_grad():
            for batch in loader:
                bi = move_to_device(batch["backend_inputs"], device)
                fused, per = self.model.get_embeddings(backend_inputs=bi, return_per_modality=True)
                fused_list.append(fused.cpu().numpy())
                if return_per_modality:
                    for m in per_mod_lists.keys():
                        if m in per:
                            per_mod_lists[m].append(per[m].cpu().numpy())

        fused_arr = np.vstack(fused_list)
        if not return_per_modality:
            return fused_arr
        per_mod = {m: np.vstack(chunks) for m, chunks in per_mod_lists.items()}
        return fused_arr, per_mod

    def __exit__(self, exc_type, exc_val, exc_tb):
        # Очистка памяти
        if hasattr(self, 'model'):
            del self.model
        if hasattr(self, 'encoders'):
            for encoder in self.encoders.values():
                if hasattr(encoder, 'model'):
                    del encoder.model
        torch.cuda.empty_cache()
        gc.collect()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 27.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 11.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 26.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 10.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires num

2025-09-02 10:46:32.435412: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756809992.688290      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756809992.762199      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Создание фиктивных данных.

In [2]:
# --- 1. Подготовка и генерация сложных данных для регрессии ---

# Установка/проверка библиотек
!pip install -q scipy transformers evaluate accelerate scikit-learn

import os
import shutil
import numpy as np
import pandas as pd
from PIL import Image
from scipy.io.wavfile import write as write_wav
from typing import List, Dict, Any, Optional

def create_complex_regression_data(
    num_samples: int = 1000,
    noise_level: float = 0.1,
    missing_rate: float = 0.05
) -> pd.DataFrame:
    """
    Создает сложный фиктивный DataFrame для демонстрации регрессии.
    Предсказываем несколько метрик видео:
    - popularity_score: 0-100 (популярность)
    - engagement_rate: 0-1 (вовлеченность)
    - quality_rating: 1-10 (оценка качества)
    - duration_minutes: 0-60 (длительность в минутах)
    """
    data_dir = "./complex_regression_data"
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)

    rng = np.random.default_rng(42)
    
    # Скрытые факторы, влияющие на целевые переменные
    content_quality = rng.uniform(0, 1, num_samples)
    production_value = rng.uniform(0, 1, num_samples)
    topic_relevance = rng.uniform(0, 1, num_samples)

    data = []
    for i in range(num_samples):
        sample = {'id': f'video_{i}'}
        
        # Базовые значения для целевых переменных (зависят от скрытых факторов)
        base_popularity = 30 * content_quality[i] + 40 * topic_relevance[i] + 20 * production_value[i]
        base_engagement = 0.3 * content_quality[i] + 0.5 * topic_relevance[i] + 0.2 * production_value[i]
        base_quality = 3 + 4 * production_value[i] + 3 * content_quality[i]
        base_duration = 5 + 30 * production_value[i] + 15 * content_quality[i]

        # --- Текстовая модальность ---
        if rng.random() > 0.1:  # 10% пропусков текста
            # Текст влияет на popularity и engagement
            text_quality = rng.uniform(0.5, 1.5)
            sample['title'] = f"Video Title {i}: {'Amazing' if text_quality > 1 else 'Standard'} Content"
            sample['description'] = f"This video covers topic {i % 20} with {'high' if text_quality > 1 else 'medium'} quality production."
            
            # Текст влияет на метрики
            base_popularity += 10 * text_quality
            base_engagement *= text_quality
        else:
            sample['title'] = None
            sample['description'] = ""

        # --- Визуальная модальность ---
        if rng.random() > 0.05:  # 5% пропусков изображений
            num_images = rng.integers(1, 6)  # от 1 до 5 кадров
            visual_quality = rng.uniform(0.7, 1.3)
            img_paths = []
            for j in range(num_images):
                path = os.path.join(data_dir, f"frame_{i}_{j}.png")
                # Создаём изображения с разной яркостью (имитация качества)
                brightness = int(128 + 50 * visual_quality)
                img = Image.fromarray(
                    rng.integers(brightness-50, brightness+50, (224, 224, 3), dtype=np.uint8)
                )
                img.save(path)
                img_paths.append(path)
            sample['keyframes'] = img_paths
            
            # Визуальное качество влияет на quality_rating и popularity
            base_quality += visual_quality
            base_popularity += 5 * visual_quality
        else:
            sample['keyframes'] = []

        # --- Аудио модальность ---
        if rng.random() > 0.15:  # 15% пропусков аудио
            num_audios = rng.integers(1, 4)  # от 1 до 3 аудиоклипов
            audio_complexity = rng.uniform(0.6, 1.4)
            audio_paths = []
            for j in range(num_audios):
                path = os.path.join(data_dir, f"audio_{i}_{j}.wav")
                sr = 48000
                duration = 1.5 * audio_complexity
                t = np.linspace(0., duration, int(sr * duration))
                amplitude = np.iinfo(np.int16).max * 0.3
                # Более сложный аудио = несколько частот
                freq1 = rng.uniform(200, 600)
                freq2 = rng.uniform(800, 1200) * audio_complexity
                waveform = amplitude * (
                    0.6 * np.sin(2. * np.pi * freq1 * t) + 
                    0.4 * np.sin(2. * np.pi * freq2 * t)
                )
                waveform = waveform.astype(np.int16)
                write_wav(path, sr, waveform)
                audio_paths.append(path)
            sample['audio_tracks'] = audio_paths
            
            # Аудио влияет на engagement и duration
            base_engagement += 0.1 * audio_complexity
            base_duration += 5 * audio_complexity
        else:
            sample['audio_tracks'] = None

        # Добавляем шум к целевым переменным
        noise = rng.normal(0, noise_level, 4)
        
        # Финальные целевые переменные с ограничениями
        sample['popularity_score'] = np.clip(base_popularity + noise[0] * 10, 0, 100)
        sample['engagement_rate'] = np.clip(base_engagement + noise[1] * 0.1, 0, 1)
        sample['quality_rating'] = np.clip(base_quality + noise[2], 1, 10)
        sample['duration_minutes'] = np.clip(base_duration + noise[3] * 5, 0, 60)
        
        # Иногда добавляем пропущенные значения
        if rng.random() < missing_rate:
            target_to_miss = rng.choice(['popularity_score', 'engagement_rate', 'quality_rating', 'duration_minutes'])
            sample[target_to_miss] = np.nan
        
        data.append(sample)

    df = pd.DataFrame(data)
    
    # Заполняем пропущенные значения медианами
    for col in ['popularity_score', 'engagement_rate', 'quality_rating', 'duration_minutes']:
        df[col].fillna(df[col].median(), inplace=True)
    
    print(f"Создан DataFrame размером {df.shape}")
    print("\nСтатистика целевых переменных:")
    target_cols = ['popularity_score', 'engagement_rate', 'quality_rating', 'duration_minutes']
    print(df[target_cols].describe())
    print("\nКорреляция между целевыми переменными:")
    print(df[target_cols].corr().round(2))
    print("\nПример строки:")
    print(df.iloc[rng.integers(0, num_samples)].to_dict())
    return df

# Генерация данных
large_regression_df = create_complex_regression_data(num_samples=50)  # Уменьшено для быстрого запуска

Создан DataFrame размером (50, 9)

Статистика целевых переменных:
       popularity_score  engagement_rate  quality_rating  duration_minutes
count         50.000000        50.000000       50.000000         50.000000
mean          57.843007         0.547682        7.266436         30.337809
std           15.695369         0.212910        1.338546          9.705532
min           12.860543         0.111881        4.766225         11.072555
25%           46.880115         0.406967        6.380709         24.031727
50%           60.992039         0.525003        7.140203         30.515140
75%           68.173288         0.659600        8.207817         36.114128
max           92.587913         1.000000       10.000000         50.589904

Корреляция между целевыми переменными:
                  popularity_score  engagement_rate  quality_rating  \
popularity_score              1.00             0.81            0.65   
engagement_rate               0.81             1.00            0.40   
qualit

/tmp/ipykernel_35/1982614490.py:133: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].median(), inplace=True)


Пример использования.

In [3]:
# --- 2. Обучение регрессионной модели ---

# Функции нормализации для целевых переменных (опционально)
def normalize_targets(targets: np.ndarray) -> np.ndarray:
    """Нормализует целевые переменные в диапазон [0, 1]"""
    # targets: [popularity_score, engagement_rate, quality_rating, duration_minutes]
    normalized = targets.copy()
    normalized[0] = targets[0] / 100.0  # popularity: 0-100 -> 0-1
    normalized[1] = targets[1]  # engagement: уже 0-1
    normalized[2] = (targets[2] - 1) / 9.0  # quality: 1-10 -> 0-1
    normalized[3] = targets[3] / 60.0  # duration: 0-60 -> 0-1
    return normalized

def denormalize_targets(normalized: np.ndarray) -> np.ndarray:
    """Денормализует предсказания обратно в исходные диапазоны"""
    targets = normalized.copy()
    targets[0] = targets[0] * 100.0  # popularity
    targets[1] = targets[1]  # engagement
    targets[2] = targets[2] * 9.0 + 1  # quality
    targets[3] = targets[3] * 60.0  # duration
    return targets

# Инициализация регрессора
video_regressor = SingleModelMultiComboRegression(
    modalities=['text', 'image', 'audio'],
    num_targets=4,  # Предсказываем 4 метрики
    target_column_names=[
        'popularity_score', 
        'engagement_rate', 
        'quality_rating', 
        'duration_minutes'
    ],
    text_columns=['title', 'description'],
    image_columns=['keyframes'],
    audio_columns=['audio_tracks'],
    
    # Конфигурация моделей
    backend='flexible',
    text_model_config={
        'checkpoint': 'microsoft/deberta-v3-small',
        'model_type': 'auto',
        'max_length': 128
    },
    image_model_config={
        'checkpoint': 'openai/clip-vit-base-patch32',
        'model_type': 'clip',
        'max_images': 3,
        'image_agg': 'mean'  # Усредняем эмбеддинги изображений
    },
    audio_model_config={
        'checkpoint': 'laion/clap-htsat-unfused',
        'model_type': 'clap',
        'max_audios': 2,
        'audio_agg': 'mean'  # Усредняем эмбеддинги аудио
    },
    
    fusion='concat',
    freeze_backbone=True,  # Замораживаем веса предобученных моделей
    
    # Опциональная нормализация
    target_normalizer=normalize_targets,
    target_denormalizer=denormalize_targets
)

# Обучение модели
video_regressor.fit(
    train_data=large_regression_df,
    epochs=2,
    test_size=0.2,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=5e-5,
    metric_names=['mse', 'mae', 'r2'],  # Множественные метрики
    loss_type='huber',  # Huber loss более устойчив к выбросам
    target_weights=[1.0, 2.0, 1.5, 0.5],  # Веса важности для каждой целевой переменной
    fp16=True,
    logging_steps=5,
    eval_steps=10,
    output_dir="./video_regressor_results",
    seed=42,
    hidden=512,
    dropout=0.2,
    fit_chunk_size=10  # Обучение по чанкам для экономии памяти
)

config.json:   0%|          | 0.00/578 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/286M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/286M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

...

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

Training Progress:   0%|          | 0/24 [00:00<?, ?step/s]

step: 10, train loss: 0.0247000000, val loss: 0.03613502532, val mse: 0.0543262362, val mae: 0.1850181818, val r2: -1.3600142285
step: 20, train loss: 0.0286000000, val loss: 0.01989620551, val mse: 0.0322626196, val mae: 0.1529631168, val r2: -0.7065489928
